In [1]:
# -*- coding: UTF-8 -*-
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.layers import Layer, LSTM, Input, Embedding, Dense, Activation, Flatten,concatenate
import tensorflow.keras.backend as K

from tensorflow.keras.activations import *

from tensorflow.keras.models import Sequential,Model

from tensorflow.keras import layers

import numpy as np

import matplotlib.pyplot as plt


from tensorflow.data import Dataset

# import keras_tuner as kt

from tensorflow.keras.layers import TextVectorization, StringLookup

import tensorflow_text as tf_text

import os
import logging

In [2]:
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.7.0


# 下载数据集

##  keras.dataset

In [ ]:
data = keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = data.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print( x_train.shape )
print( x_test.shape )
print( y_train.shape )
print( y_test.shape )

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

class_names[y_train[0]]

In [ ]:
plt.imshow(x_train[0])

##  tensorflow_datasets

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Construct a tf.data.Dataset
ds = tfds.load('mnist', split='train', shuffle_files=True)

# Build your input pipeline
# ds = ds.shuffle(1024).batch(32).prefetch(tf.data.AUTOTUNE)

for example in ds.take(1):
  image, label = example["image"], example["label"]

In [ ]:
print(image.shape)

# image

## hugging dataset

In [ ]:
from datasets import load_dataset

from tokenizers import Tokenizer
from tokenizers.processors import TemplateProcessing
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase, NFD, StripAccents
from tokenizers.trainers import BpeTrainer, WordLevelTrainer
from tokenizers.models import WordLevel, BPE
from tokenizers.pre_tokenizers import Whitespace,WhitespaceSplit


In [ ]:
config=dict(
    # RUN CONFIG:
    RUN_NAME='unofficial_single_gpu_run',
    RUN_DESCRIPTION='No description',
    RUNS_FOLDER_PTH='../runs',
    # DATA CONFIG:
    DATASET_SIZE=30000, # WARN: This is too small but makes it easy to go through the notebook fast
    TEST_PROPORTION=0.01,
    MAX_SEQ_LEN=40,
    VOCAB_SIZE=60000,
    TOKENIZER_TYPE='wordlevel',
    # TRAINING CONFIG:
    BATCH_SIZE=48, 
    GRAD_ACCUMULATION_STEPS=2048//48,
    WORKER_COUNT=10,
    EPOCHS=100,
    # OPTIMIZER CONFIG:
    BETAS=(0.9, 0.98),
    EPS=1e-9,
    # SCHEDULER CONFIG:
    N_WARMUP_STEPS=4000, 
    # MODEL CONFIG:
    D_MODEL=512,
    N_BLOCKS=6,
    N_HEADS=8,
    D_FF=2048,
    DROPOUT_PROBA=0.1,
    # OTHER:
    MODEL_SAVE_EPOCH_CNT=10,
    DEVICE='gpu',
    LABEL_SMOOTHING=0.1,
)

In [ ]:
from datasets import load_dataset_builder
dataset_builder = load_dataset_builder('wmt14', 'de-en')
print(dataset_builder.cache_dir)
print(dataset_builder.info.features)
print(dataset_builder.info.splits)

In [ ]:
cache_dir='dataset/wmt14/de-en/1.0.0'

In [ ]:
data=load_dataset('wmt14','de-en',split='train')#.shuffle(seed=42)

# data=load_dataset('dataset/wmt14/de-en/1.0.0',split='train')

data=data.select(range(30000)) 
data=data.flatten()
data=data.rename_column('translation.de','translation_trg')
data=data.rename_column('translation.en','translation_src')

# 数据预处理

## 操作 tensor

### 转换 Numpy

In [ ]:
tf.constant(10)

tf.constant(10).numpy()

### 累加

In [ ]:
a = tf.constant(1)

a += 1

a

### reshape

In [ ]:
a =  [[[1, 3], [2, 3]],
        [[2, 1], [1, 2]],
        [[3, 3], [3, 2]]]


# a.numpy()


tf.shape(a)

b = tf.reshape(a, [-1,2])

b.numpy()


### 按照索引取元素

#### tf.gather

In [3]:
a=np.array([1,2,3,4])

a[[0,2]]

array([1, 3])

In [4]:
a = np.array(['ab the','b','c'])

a

list(a)

array(['ab the', 'b', 'c'], dtype='<U6')

['ab the', 'b', 'c']

In [5]:
a = np.array(['a','b','c'])

a[[0,1]]

array(['a', 'b'], dtype='<U1')

In [ ]:
a = tf.constant(['a','b','c'])

a[0]

# a[[0,1]]

In [6]:
params = tf.constant(['p0', 'p1', 'p2', 'p3', 'p4', 'p5'])
params[3].numpy()

tf.gather(params, 3).numpy()


b'p3'

b'p3'

In [7]:
indices = [2, 0, 2, 5]
tf.gather(params, indices).numpy()


array([b'p2', b'p0', b'p2', b'p5'], dtype=object)

In [8]:
params = tf.constant([[0, 1.0, 2.0],
                      [10.0, 11.0, 12.0],
                      [20.0, 21.0, 22.0],
                      [30.0, 31.0, 32.0]])

tf.gather(params, indices=[3,1]).numpy()


tf.gather(params, indices=[2,1], axis=1).numpy()




array([[30., 31., 32.],
       [10., 11., 12.]], dtype=float32)

array([[ 2.,  1.],
       [12., 11.],
       [22., 21.],
       [32., 31.]], dtype=float32)

#### tf.nn.embedding_lookup

In [13]:

params = tf.constant([[0, 1.0, 2.0],
                      [10.0, 11.0, 12.0],
                      [20.0, 21.0, 22.0],
                      [30.0, 31.0, 32.0]])


ids = [0, 1, 2]

out = tf.nn.embedding_lookup(params, ids)

out

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 0.,  1.,  2.],
       [10., 11., 12.],
       [20., 21., 22.]], dtype=float32)>

In [15]:

ids = tf.constant( [
                    [0, 1],
                    [1, 2]
                    ])

out = tf.nn.embedding_lookup(params, ids)

out


<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[ 0.,  1.,  2.],
        [10., 11., 12.]],

       [[10., 11., 12.],
        [20., 21., 22.]]], dtype=float32)>

### 数组 (list append)

In [ ]:

# ta = tf.TensorArray(tf.float32, size=3, clear_after_read=False)

ta = tf.TensorArray(tf.float32, size=3)


ta = ta.write(0, 10)
ta = ta.write(1, 20)
ta = ta.write(2, 30)

ta.read(0)

ta.read(1)

ta.read(2)

ta.stack()



In [ ]:

array = tf.TensorArray(tf.float32, size=0, dynamic_size=True)

slice0 = tf.ones((3,))

array = array.write(0, slice0)


array.stack()

In [ ]:
embed = Embedding(20,10)

x = embed(array.stack())

x

In [ ]:

x[:, -1, :]




### 切片赋值


In [ ]:
batch_size, time, width, height, channels = 13,11,7,5,3
video_batch = tf.zeros([batch_size, time, width, height, channels])

indices = [[0],[1]]
new_clips = tf.ones([2, time, width, height, channels])


# tf.tensor_scatter_nd_update(video_batch, indices, new_clips)

In [ ]:
indices = [[0, 0], [1, 0], [2, 0]] # num_updates=3, index_depth=2
new_images = tf.ones([
  # num_updates=3, inner_shape=(width, height, channels)
  3, width, height, channels])
# tf.tensor_scatter_nd_update(video_batch, indices, new_images)

In [ ]:
N_batch = tf.constant(2)

target_length = 3

n_vocab_target = 4 


probs = tf.zeros((int(target_length), int(N_batch), n_vocab_target))

indices = [[1]]

new_clips = tf.ones([1, N_batch, n_vocab_target])

probs_new =  tf.tensor_scatter_nd_update(probs, indices, new_clips)

probs_new

In [ ]:
# my_var = my_var[4:8].assign(tf.zeros(4))

### one hot 相互转换

In [ ]:
indices = [[0, 2, 0], 
           [1, 0, 1]]
depth = 3

one_hot = tf.one_hot(indices, depth,
           on_value=1, off_value=0,  dtype=tf.int64,
           axis=-1)

one_hot

In [ ]:
dense = tf.argmax(one_hot, axis=-1)

dense

### tf.where

In [ ]:
tf.where([True, False, False, True], [1,2,3,4], [100,200,300,400])


tf.where([True, False, False, True], [1,2,3,4], [100])


tf.where([True, False, False, True], [1,2,3,4], 100)


tf.where([True, False, False, True], 1, 100)



## tf.data 数据流水线 ( Dataset )


### 分批次遍历 (batch)

In [ ]:
# Two tensors can be combined into one Dataset object.
features = tf.constant([[1, 3], 
                        [2, 1], 
                        [3, 3]]) # ==> 3x2 tensor
labels = tf.constant(['A', 'B', 'A']) # ==> 3x1 tensor
dataset = Dataset.from_tensor_slices((features, labels))  # 把tensor 的第一个维度作为样本 id 维度, 按照此维度拼接样本

list(dataset.as_numpy_iterator()) 

In [ ]:
# Both the features and the labels tensors can be converted
# to a Dataset object separately and combined after.
features_dataset = Dataset.from_tensor_slices(features)
labels_dataset = Dataset.from_tensor_slices(labels)
dataset = Dataset.zip((features_dataset, labels_dataset))

list(dataset.as_numpy_iterator())

In [ ]:
# A batched feature and label set can be converted to a Dataset
# in similar fashion.
batched_features = tf.constant([[[1, 3], 
                                 [2, 3]],
                                [[2, 1], 
                                 [1, 2]],
                                [[3, 3], 
                                 [3, 2]]], shape=(3, 2, 2))
batched_labels = tf.constant([['A', 'A'],
                              ['B', 'B'],
                              ['A', 'B']], shape=(3, 2))
dataset = Dataset.from_tensor_slices((batched_features, batched_labels))

dataset_batch2 = dataset.batch(2)

for i,element in enumerate(dataset):
    
    print(i)
    
    print(element[0])
    
#     print(element[1].shape)
    
    print(element[1].numpy())

In [ ]:
for i,element in enumerate(dataset_batch2):
    
    print(i)
    print(element)
    


In [ ]:

dataset_batch1 = dataset.batch(1)

for i,element in enumerate(dataset_batch1):
    
    print(i)
    
    print(element)
    


In [ ]:
dataset_unbatch = dataset_batch2.unbatch()

for i,element in enumerate(dataset_unbatch):
    
    print(i)
    
    print(element)

In [ ]:
label2 = tf.constant([[1],
                      [2],
                      [3]])


for feature,label,label2 in zip(batched_features, batched_labels, label2):
    
    print(feature)
    
    print(label)
    
    print(label2)

### 分批次映射 (map)

In [ ]:
features = tf.constant( [
                        [1, 3, 2, 3],
                        [2, 1, 1, 2],
                        [3, 3, 3, 2],
                        [0, 0, 0, 2]
                        ], shape=(4, 4))

dataset = Dataset.from_tensor_slices(features).batch(2) # 后面每一次遍历的单位都是 batch 

source_in = dataset.map(lambda batch_text: batch_text[:, ::-1], num_parallel_calls=tf.data.AUTOTUNE)

target_out = dataset.map(lambda batch_text: batch_text[:, 1:], num_parallel_calls=tf.data.AUTOTUNE)

target_in = dataset.map(lambda batch_text: batch_text[:, :-1], num_parallel_calls=tf.data.AUTOTUNE)


print('source_in')

for ele in source_in:
    
    print(ele)


print('target_out')

for ele in target_out:
    
    print(ele)

print('target_in')
    
for ele in target_in:
    
    print(ele)
    



### 访问列 

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices([[1,2,3,4], 
                                              [5,6,7,8]])
dataset_filter = dataset.map(lambda x: tf.gather(x, [0, 2], axis=0))
result = list(dataset_filter.as_numpy_iterator())
print(result)

In [ ]:

features = tf.constant( [
                        [1, 3, 2, 3],
                        [2, 1, 1, 2],
                        [3, 3, 3, 2],
                        [0, 0, 0, 2]
                        ], shape=(4, 4))

labels = tf.constant(['A', 'B', 'A', 'A'])

dataset = Dataset.from_tensor_slices((features, labels)).batch(2)

slice0 = dataset.map(lambda batch_feature, batch_label : batch_feature)

slice1 = dataset.map(lambda batch_feature, batch_label : batch_label)

for ele in slice0:
    
    print(ele)

### 按照序列长度划分 batch

ref:

[1] https://www.tensorflow.org/api_docs/python/tf/data/Dataset#bucket_by_sequence_length

[2] https://www.jianshu.com/p/2e83d11c7ae6

In [ ]:
import six

def example_length(example):
  length = 0
  # Length of the example is the maximum length of the feature lengths
  for _, v in sorted(six.iteritems(example)):
    # For images the sequence length is the size of the spatial dimensions.
    feature_length = tf.shape(v)[0]
    if len(v.get_shape()) > 2:
      feature_length = tf.shape(v)[0] * tf.shape(v)[1]
    length = tf.maximum(length, feature_length)
  return length

In [ ]:
def _bucket_boundaries(max_length, min_length=8, length_bucket_step=1.1):
  """A default set of length-bucket boundaries."""
  assert length_bucket_step > 1.0
  x = min_length
  boundaries = []
  while x < max_length:
    boundaries.append(x)
    x = max(x + 1, int(x * length_bucket_step))
  return boundaries


def batching_scheme(batch_size,
                    max_length,
                    min_length_bucket,
                    length_bucket_step,
                    drop_long_sequences=False,
                    shard_multiplier=1,
                    length_multiplier=1,
                    min_length=0):
  """A batching scheme based on model hyperparameters.

  Every batch contains a number of sequences divisible by `shard_multiplier`.

  Args:
    batch_size: int, total number of tokens in a batch.
    max_length: int, sequences longer than this will be skipped. Defaults to
      batch_size.
    min_length_bucket: int
    length_bucket_step: float greater than 1.0
    drop_long_sequences: bool, if True, then sequences longer than
      `max_length` are dropped.  This prevents generating batches with
      more than the usual number of tokens, which can cause out-of-memory
      errors.
    shard_multiplier: an integer increasing the batch_size to suit splitting
      across datashards.
    length_multiplier: an integer multiplier that is used to increase the
      batch sizes and sequence length tolerance.
    min_length: int, sequences shorter than this will be skipped.

  Returns:
     A dictionary with parameters that can be passed to input_pipeline:
       * boundaries: list of bucket boundaries
       * batch_sizes: list of batch sizes for each length bucket
       * max_length: int, maximum length of an example

  Raises:
    ValueError: If min_length > max_length
  """
  max_length = max_length or batch_size
  if max_length < min_length:
    raise ValueError("max_length must be greater or equal to min_length")

  boundaries = _bucket_boundaries(max_length, min_length_bucket,
                                  length_bucket_step)
  boundaries = [boundary * length_multiplier for boundary in boundaries]
  max_length *= length_multiplier

  batch_sizes = [
      max(1, batch_size // length) for length in boundaries + [max_length]
  ]
  max_batch_size = max(batch_sizes)
  # Since the Datasets API only allows a single constant for window_size,
  # and it needs divide all bucket_batch_sizes, we pick a highly-composite
  # window size and then round down all batch sizes to divisors of that window
  # size, so that a window can always be divided evenly into batches.
  # TODO(noam): remove this when Dataset API improves.
  highly_composite_numbers = [
      1, 2, 4, 6, 12, 24, 36, 48, 60, 120, 180, 240, 360, 720, 840, 1260, 1680,
      2520, 5040, 7560, 10080, 15120, 20160, 25200, 27720, 45360, 50400, 55440,
      83160, 110880, 166320, 221760, 277200, 332640, 498960, 554400, 665280,
      720720, 1081080, 1441440, 2162160, 2882880, 3603600, 4324320, 6486480,
      7207200, 8648640, 10810800, 14414400, 17297280, 21621600, 32432400,
      36756720, 43243200, 61261200, 73513440, 110270160
  ]
  window_size = max(
      [i for i in highly_composite_numbers if i <= 3 * max_batch_size])
  divisors = [i for i in range(1, window_size + 1) if window_size % i == 0]
  batch_sizes = [max([d for d in divisors if d <= bs]) for bs in batch_sizes]
  window_size *= shard_multiplier
  batch_sizes = [bs * shard_multiplier for bs in batch_sizes]
  # The Datasets API splits one window into multiple batches, which
  # produces runs of many consecutive batches of the same size.  This
  # is bad for training.  To solve this, we will shuffle the batches
  # using a queue which must be several times as large as the maximum
  # number of batches per window.
  max_batches_per_window = window_size // min(batch_sizes)
  shuffle_queue_size = max_batches_per_window * 3

  ret = {
      "boundaries": boundaries,
      "batch_sizes": batch_sizes,
      "min_length": min_length,
      "max_length": (max_length if drop_long_sequences else 10**9),
      "shuffle_queue_size": shuffle_queue_size,
  }
  return ret

batch_size = 4096
max_length = 256
min_length_bucket= 8
length_bucket_step= 1.1

rt = batching_scheme(batch_size,max_length,min_length_bucket, length_bucket_step)

rt['boundaries']
rt['batch_sizes']

#### 从生成器(from_generator)中构建

In [ ]:
elements = [
  [0], 
  [1, 2, 3, 4], 
  [5, 6, 7],
  [7, 8, 9, 10, 11], 
  [13, 14, 15, 16, 19, 20], 
  [21, 22]]

dataset = tf.data.Dataset.from_generator(
    lambda: elements, tf.int32, output_shapes=[None])

for batch in dataset:
    print(batch)

# dataset = dataset.bucket_by_sequence_length(
#         element_length_func=lambda elem: tf.shape(elem)[0],
#         bucket_boundaries=[3, 5],
#         bucket_batch_sizes=[2, 2, 2]) # bucket_batch_sizes 定义了各个 batch 的比例大小(不是真实大小)

dataset = dataset.bucket_by_sequence_length(
        element_length_func=lambda elem: tf.shape(elem)[0],
        bucket_boundaries=[3, 5],
        bucket_batch_sizes=[4, 4, 4]) # bucket_batch_sizes 定义了各个 batch 的比例大小(不是真实大小)

for batch in dataset:
    print(batch)


# tf.Tensor(
# [[1 2 3 4]
#  [5 6 7 0]], shape=(2, 4), dtype=int64)

# tf.Tensor(
# [[ 7  8  9 10 11  0]
#  [13 14 15 16 19 20]], shape=(2, 6), dtype=int64)

# tf.Tensor(
# [[ 0  0]
#  [21 22]], shape=(2, 2), dtype=int64)
    
# bucket_boundaries=[3, 5] : batch 中的序列的长度分别为 2, 4, 6
# 2 满足第一个区间的长度要求 (小于3)
# 4 满足第二个区间的长度要求 (大于等于3 小于5)
# 6 满足第二个区间的长度要求 (大于等于5 )

In [ ]:
elements1 = [
  [0], 
  [1, 2, 3, 4], 
  [5, 6, 7],
  [7, 8, 9, 10, 11], 
  [13, 14, 15, 16, 19, 20], 
  [21, 22]
]

elements2 = [
  [0,99], 
  [1, 2, 3, 4], 
  [5, 6, 7],
  [7, 8, 9, 10, 11], 
  [13, 14, 15, 16, 19, 20], 
  [21, 22]
]

dataset1 = tf.data.Dataset.from_generator(
    lambda: elements1, tf.int32, output_shapes=[None])

dataset2 = tf.data.Dataset.from_generator(
    lambda: elements2, tf.int32, output_shapes=[None])


dataset = tf.data.Dataset.zip((dataset1, dataset2))


dataset = dataset.bucket_by_sequence_length(
        element_length_func=lambda ele1, ele2: tf.shape(ele1)[0],
        bucket_boundaries=[3, 5],
        bucket_batch_sizes=[2, 2, 2])

# dataset = dataset.bucket_by_sequence_length(
#         element_length_func=lambda ele1, ele2: tf.shape(ele1)[0],
#         bucket_boundaries= rt['boundaries'],
#         bucket_batch_sizes= rt['batch_sizes'])


for batch in dataset:
    print(batch)

#### Dataset 转换为生成器

In [ ]:
elements1 = [
  [0], 
  [1, 2, 3, 4], 
  [5, 6, 7],
  [7, 8, 9, 10, 11], 
  [13, 14, 15, 16, 19, 20], 
  [21, 22]
]

elements2 = [
  [0,99], 
  [1, 2], 
  [5, 6, 7],
  [7, 8, 9, 10, 11, 12, 13], 
  [13, 14, 15, 16], 
  [21, 22, 23]
]

ragged1 = tf.ragged.constant(elements1)
ragged2 = tf.ragged.constant(elements2)

dataset1 = Dataset.from_tensor_slices(ragged1)
dataset2 = Dataset.from_tensor_slices(ragged2)

dataset = tf.data.Dataset.zip((dataset1, dataset2))

list(dataset)[0]


dataset = dataset.map(lambda x,y: (x,y)) # 转换为生成器

dataset_backet = dataset.bucket_by_sequence_length(
        element_length_func=lambda x,y: tf.cast(tf.maximum(tf.shape(x)[0],tf.shape(y)[0]), tf.int32),
        bucket_boundaries=[3, 5],
        bucket_batch_sizes=[4, 4, 4],
        )

# bucket_fn = tf.data.experimental.bucket_by_sequence_length(
#     lambda x,y: tf.cast(tf.shape(x)[0], tf.int32), 
#     bucket_boundaries=[3, 5], 
#     bucket_batch_sizes=[1, 4, 4], 
# )

# dataset_backet = dataset.map(lambda x,y: (x,y)).apply(bucket_fn)


for data in dataset_backet:
    
    print(data)


In [ ]:
def load_corpus_data( corpus_file_dir):
    """
    读取 图片描述文本, 并将它们和对应的图片进行映射

    :param corpus_file_dir: 语料文本

    :return:

    text_data: 所有句子的列表

    """

    with open(corpus_file_dir, encoding='utf-8') as corpus_file:

        lines = corpus_file.readlines()
        text_data = []

        for line in lines:
#             sentence = line.rstrip("\n")

            sentence = line.strip()

            sentence = '[START] ' + sentence + ' [END]'

            text_data.append(sentence)

        return text_data
    
corpus = load_corpus_data('dataset/newstest2012.de')
corpus_dataset = Dataset.from_tensor_slices(corpus).batch(2)

In [ ]:

corpus_dataset_row = corpus_dataset.unbatch()

list(corpus_dataset_row)[0]

list(corpus_dataset_row)[0].numpy()


In [ ]:
model_tokenizer = SpaceTokenizer(corpus) 

corpus_vector = corpus_dataset.map(lambda batch_text: model_tokenizer.tokenize(batch_text))

list(corpus_vector)[0]

corpus_vector_row = corpus_vector.unbatch()

list(corpus_vector_row)[0]


In [ ]:
tf.shape(list(corpus_vector_row)[0])[0]

In [ ]:

# corpus_vector_backet = corpus_vector_row.bucket_by_sequence_length(
#         element_length_func=lambda ele1: tf.shape(ele1)[0],
#         bucket_boundaries= rt['boundaries'],
#         bucket_batch_sizes= rt['batch_sizes'])

bucket_fn = tf.data.experimental.bucket_by_sequence_length(
    lambda x: tf.cast(tf.shape(x)[0], tf.int32), 
    bucket_boundaries=rt['boundaries'], 
    bucket_batch_sizes=rt['batch_sizes'], 
)

# Apply bucketing
corpus_vector_backet = corpus_vector_row.map(lambda x: x).apply(bucket_fn)


for batch in corpus_vector_backet.take(3):
    print(batch)

### 混洗(shuffle)

In [ ]:
dataset = tf.data.Dataset.range(3)
dataset = dataset.shuffle(3)
list(dataset.as_numpy_iterator())
# [1, 0, 2]
list(dataset.as_numpy_iterator())
# [1, 2, 0]

In [ ]:
dataset = tf.data.Dataset.range(3)
dataset = dataset.shuffle(3, reshuffle_each_iteration=False)
list(dataset.as_numpy_iterator())
# [1, 0, 2]
list(dataset.as_numpy_iterator())
# [1, 0, 2]

In [ ]:
features = tf.constant( [
                        [1, 3, 2, 3,1,1,1],
                        [2, 1, 1, 2,0,0,0],
    
                        [3, 3, 3, 2,2],
                        [0, 0, 0, 2,0],
    
                        [1, 3, 2, 3,3,3],
                        [2, 1, 1, 2,2,3],
    
                        [3, 3, 3, 2],
                        [0, 0, 0, 2]
    
                        ], shape=(8, None))

dataset = Dataset.from_tensor_slices(features).shuffle(4).batch(2)  



for i,ele in enumerate(dataset):
    
    print(i)
    
    print(ele)

for i,ele in enumerate(dataset):
    
    print(i)
    
    print(ele)
    

In [ ]:
features = tf.constant( [
                        [1, 3, 2, 3],
                        [2, 1, 1, 2],
                        [3, 3, 3, 2],
                        [0, 0, 0, 2]
                        ], shape=(4, 4))

dataset = Dataset.from_tensor_slices(features).shuffle(4).batch(2) 

target_out = dataset.map(lambda batch_text: batch_text[:, 1:], num_parallel_calls=tf.data.AUTOTUNE)

target_in = dataset.map(lambda batch_text: batch_text[:, :-1], num_parallel_calls=tf.data.AUTOTUNE)

print('target_out')
# 每次遍历都会打乱顺序
for ele in target_out:
    
    print(ele)

print('target_in')
    
for ele in target_in:
    
    print(ele)

    
# target_out
# tf.Tensor(
# [[3 2 3]
#  [0 0 2]], shape=(2, 3), dtype=int32)

# target_in
# tf.Tensor(
# [[2 1 1]
#  [0 0 0]], shape=(2, 3), dtype=int32)

# 元素不再一一对应

In [ ]:

features = tf.constant( [
                        [1, 3, 2, 3],
                        [2, 1, 1, 2],
                        [3, 3, 3, 2],
                        [0, 0, 0, 2]
                        ], shape=(4, 4))

labels = tf.constant(['A', 'B', 'C', 'D'])

dataset = Dataset.from_tensor_slices((features, labels)).batch(2)

dataset = dataset.unbatch().shuffle(4).batch(2)


for ele in dataset:
    
    print(ele)


for ele in dataset:
    
    print(ele)

    

### 两个数据集连接

In [4]:
a = tf.data.Dataset.range(1, 4)  # ==> [ 1, 2, 3 ]
b = tf.data.Dataset.range(4, 8)  # ==> [ 4, 5, 6, 7 ]
ds = a.concatenate(b)
list(ds.as_numpy_iterator())

# The input dataset and dataset to be concatenated should have
# compatible element specs.
c = tf.data.Dataset.zip((a, b))
a.concatenate(c)



d = tf.data.Dataset.from_tensor_slices(["a", "b", "c"])
a.concatenate(d)




[1, 2, 3, 4, 5, 6, 7]

TypeError: Incompatible types of input datasets: <dtype: 'int64'> vs. (tf.int64, tf.int64).

##  使用 tensorflow_text 文本预处理工具 

### 字符串 (Strings)


In [ ]:
text_utf8 = tf.constant(u"中")

text_utf8

In [ ]:
# unicode 编码
hex(ord('中'))

# utf-8 编码
'中'.encode('utf-8')


In [ ]:
text_utf8 = tf.constant("语言处理")

text_utf8

text_utf8.numpy().decode('utf-8')

### 正则表达式

In [ ]:
ord('„') # unicode 编码

ord('“')

chr(8220)

#### 匹配 unicode 范围

**Ref**

[1] https://segmentfault.com/a/1190000021141670

[2] https://github.com/google/re2/wiki/Syntax#perl

In [ ]:
"""
[^**]	表示不匹配此字符集中的任何一个字符
\u4e00-\u9fa5	汉字的unicode范围
\u0030-\u0039	数字的unicode范围
\u0041-\u005a	大写字母unicode范围
\u0061-\u007a	小写字母unicode范围
\u0021-\u007e   英文字母unicode范围(含数字与符号)
\u00C0-\u00FF   德文/法文的unicode范围
\uAC00-\uD7AF	韩文的unicode范围
\u3040-\u31FF	日文的unicode范围

"""

In [ ]:
text =' a b c , d . , '

remove_unk = r'[^[:alnum:]]' 
# [[:alnum:]]  (≡ [0-9A-Za-z] )

text = tf.strings.regex_replace(text, remove_unk, ' ')

print(text)

In [ ]:
import re 

text = ['我 Ämter Aushärtung für Autostraßen Öfen , abc-10 Pro 50,813 words and  “ format articles „ .']


# 删除在各个语系外的字符
remove_unk =   r'[^\p{Latin}|[:print:]]' 

# \p{Han} 匹配中文语系
# \p{Latin} 匹配拉丁语系
# [[:alnum:]]  (≡ [0-9A-Za-z] )
# [[:print:]]  (≡ [A-Za-z0-9!"#$%&'()*+,\-./:;<=>?@[\\\]^_`{|}~] )

# ref
# [1] https://segmentfault.com/a/1190000021141670
# [2] https://github.com/google/re2/wiki/Syntax#perl


text = tf.strings.regex_replace(text, remove_unk, ' ')


print(text[0])

print(text[0].numpy().decode('utf-8'))

#### 匹配独立出现的数字

In [ ]:
text = '11 999 avc 10 abc-10 10-abc abc10 50 813 aa 4x4 20'


#  A=( \d+) 空格+至少1个数字 为匹配字符A ,
#  B='A+ ' 至少1个A + 空格 为匹配字符B
# remove_digits = r'( \d+)+ '

# 考虑数字可能出现在句首
# remove_digits = r'^(\d+ )+|( \d+)+ '

# 考虑数字可能出现在句首和句末 
remove_digits = r'^(\d+ )+|( \d+)+ |(\d+)$'

text = tf.strings.regex_replace(text, remove_digits, ' ')

print(text)


#### 去除控制字符

In [3]:

import re 

_null_str=''
_start_str=re.escape('[START]') 
_end_str=re.escape('[END]')
_unk_str=re.escape('[UNK]')


remove_word_re = re.compile(
            r'{}|{}|{}'.format(_null_str, _start_str, _end_str))


text = '[START] A group of race horses [UNK] [UNK]  run down a track carrying jockeys  [END]'

text_clean = remove_word_re.sub('', text)

text_clean



' A group of race horses [UNK] [UNK]  run down a track carrying jockeys  '

In [ ]:
remove_word_re = re.compile(
            r'{}|{}|{}|{}'.format(_null_str, _start_str, _end_str, _unk_str))

text_clean = remove_word_re.sub('', text)


text_clean.split()

In [ ]:
text2 = '[UNK] und [UNK] [UNK] immer noch [UNK] [END]'

text2_clean = remove_word_re.sub('', text2)

text2_clean

#### 去除匹配子串中的空格

In [9]:

import re 

dash_str = '# # AT # # - # # AT # #' # 破折号
double_quotation_str = '& quot ;' # 双引号
quotation_str = '& apos ;' # 单引号


dash_re = re.compile(r'{}'.format(re.escape(dash_str)))

double_quotation_re = re.compile(r'{}'.format(re.escape(double_quotation_str)))

quotation_re = re.compile(r'{}'.format(re.escape(quotation_str)))


In [16]:
text = 'EU # # AT # # - # # AT # # Botschafterin & quot ; Ich wurde stets & quot ; & apos ; Obani '

text = dash_re.sub(dash_str.replace(' ', ''), text)

text = double_quotation_re.sub(double_quotation_str.replace(' ', ''), text)

text = quotation_re.sub(quotation_str.replace(' ', ''), text)

text

'EU ##AT##-##AT## Botschafterin &quot; Ich wurde stets &quot; &apos; Obani '

### 文本标准化

ref: 

[1]https://www.tensorflow.org/text/api_docs/python/text/normalize_utf8

[2] https://www.elastic.co/guide/cn/elasticsearch/guide/current/unicode-normalization.html


##### NFC 和 NFD

In [ ]:
import unicodedata


s1='\u00f1'                                                            
 
s2='n\u0303'                                                            
 
print(s1)
print(s2)

s1 == s2
                                                              

In [ ]:
a= b'schl\xc3\xa4gt'

print(a.decode('utf-8'))


b = 'schlägt'
print(b)

a==b

**第一种 \u00f1 为整体表示法 (NFC)**

**第二种 n\u0303 为组合表示法，是n和字符~的组合字符 (NFD)**

In [ ]:
t1 = unicodedata.normalize('NFC', s1)
t2 = unicodedata.normalize('NFC', s2)

t1.encode('raw_unicode_escape')

t2.encode('raw_unicode_escape')

t1==t2  

In [ ]:
t3 = unicodedata.normalize('NFD', s1)
t4 = unicodedata.normalize('NFD', s2)

t3.encode('raw_unicode_escape')

t4.encode('raw_unicode_escape')

t3==t4  


In [ ]:
print(t1)
print(t3)

t1==t3

t1.encode('utf-8')

t3.encode('utf-8')

In [ ]:
a = 'Aushärtung für Autostraßen Öfen nämlich'

print(a.encode('utf-8'))

normal_nfkd = tf_text.normalize_utf8(a, 'NFKD')

print('nfkd: ',normal_nfkd)

normal_nfkc = tf_text.normalize_utf8(a, 'NFKC')

print('nfkc:', normal_nfkc)


print(normal_nfkd.numpy().decode('utf-8'))

# b = 'Weißkopfseeadler WEISSKOPFSEEADLER'

# normal_b = tf_text.normalize_utf8(b, 'NFKD')

# print(normal_b.numpy().decode('utf-8'))


In [ ]:
# ["株式会社", "ＫＡＤＯＫＡＷＡ"]

a = "ＫＡＤＯＫＡＷＡ"

print(a.encode('utf-8'))

normal_a = tf_text.normalize_utf8(a)

print(normal_a)

print(normal_a.numpy().decode('utf-8'))


##### 小写化

In [ ]:
text = ['Ämter Aushärtung für Autostraßen Öfen Weißkopfseeadler WEISSKOPFSEEADLER ',
        'Dadurch sind die Hiesigen eine Sorge los - nämlich die Vertreibung der Kreuzfahrer ']

text = tf_text.normalize_utf8(text, 'NFKD')

# text = tf_text.normalize_utf8(text, 'NFKC')

text = tf.strings.lower(text)

text

text.numpy()[0].decode('utf-8')


##### 大小写折叠

lower函数只支持ascill表中的字符，而casefold则支持很多不同种类的语言。
比如说β，lower 只能显示出原形而casefold则能显示他的小写--ss

In [ ]:
s1='Autostraßen'

s1.lower()

s1.casefold()


In [ ]:
text = ['Ämter Aushärtung für Autostraßen Öfen Weißkopfseeadler WEISSKOPFSEEADLER ']

text = tf_text.case_fold_utf8(text)

text

text.numpy()[0].decode('utf-8')



In [ ]:
s1='\u00f1'                                                            
 
s2='n\u0303'                                                            
 
print(s1)
print(s2)

print(s1.encode('utf-8'))

print(s2.encode('utf-8'))

text = ['{} - {}'.format(s1, s2)]

text = tf_text.case_fold_utf8(text)

text

# text.numpy()[0].decode('utf-8')

### 最终结果

In [ ]:
import re 
import string

text = '„ &apos 我 English &quot ##AT##-##AT## abc10 abc-10 4x4  ,, Weißkopfseeadler Ämter Aushärtung für Autostraßen Öfen ,  50,813 words “   .'


# 删除单词的规则
remove_words = r'(##AT##-##AT##|&apos|&quot)'

# 删除在各个语系外的字符
remove_unk =   r'[^\p{Latin}|[:print:]]' 

# \p{Latin} 匹配拉丁语系
# [[:print:]]  (≡ [A-Za-z0-9!"#$%&'()*+,\-./:;<=>?@[\\\]^_`{|}~] )


# 需要删除的标点符号, 排除 "'" 和 "-"
punctuation = string.punctuation
punctuation = punctuation.replace("'", "")  # 英语中有', 不删除 '
punctuation = punctuation.replace("-", "")  #
remove_punc = r'[%s]' % re.escape(punctuation)

# 删除单独的数字字符
# eg. 'avc 10 abc-10 abc10' -> 'avc  abc-10 abc10'

remove_digits = r'^(\d+ )+|( \d+)+ |(\d+)$'


# 删除在各个语系外的字符
text = tf.strings.regex_replace(text, remove_unk, '')

# 清除指定单词
text = tf.strings.regex_replace(text, remove_words, '')

# NKFC unicode 标准化 +  大小写折叠
text = tf_text.case_fold_utf8(text)

# 清除句子中的标点符号
text = tf.strings.regex_replace(text, remove_punc, ' ')  # 空1格

# 清除句子中的独立数字
text = tf.strings.regex_replace(text, remove_digits, '')  

# 清除左右端点的空格
text = tf.strings.strip(text)

print(text)

print(text.numpy().decode('utf-8'))

## 分词器

### TextVectorization

In [ ]:
def load_corpus_data( corpus_file_dir):
    """
    读取 图片描述文本, 并将它们和对应的图片进行映射

    :param corpus_file_dir: 语料文本

    :return:

    text_data: 所有句子的列表

    """

    with open(corpus_file_dir, encoding='utf-8') as corpus_file:

        lines = corpus_file.readlines()
        text_data = []

        for line in lines:
#             sentence = line.rstrip("\n")

            sentence = line.strip()

            sentence = '[START] ' + sentence + ' [END]'

            text_data.append(sentence)

        return text_data
    
corpus = load_corpus_data('dataset/newstest2012.de')

In [ ]:
corpus[:2]

In [ ]:
from collections import Counter

def statistic_seq_length( seq_list):
    """
    统计序列的长度

    :param seq_list:
    :return:
    """

    seq_length = {}

    for i, seq in enumerate(seq_list):

        seq_arr = seq.split(" ")

        seq_length[i] = len(seq_arr)

    return seq_length

seq_length = statistic_seq_length(corpus)
# seq_length = { 句子标号: 句子的长度,  }

seq_length_counter =  Counter(seq_length.values())
# seq_length_counter = { 句子的长度: 该长度句子的个数 }

# seq_length_counter.most_common(10)

# sorted(seq_length_counter.items(), key=lambda d: d[0], reverse=True)

In [ ]:

corpus = tf_text.normalize_utf8(corpus)

corpus = tf.strings.lower(corpus)


n_vocab = 20000

tokenizer = TextVectorization(
            standardize=None,
            output_sequence_length=50,
            max_tokens=n_vocab)

tokenizer.adapt(corpus)

vocab_list = tokenizer.get_vocabulary()


In [ ]:
corpus[:2]

In [ ]:
vocab_list[:10]

# vocab_list.index('Öfen')

In [ ]:
# text = ['Keine befreiende Novelle für Tymoshenko durch das Parlament']

text = ['Keine befreiende Novelle für Tymoshenko durch das Parlament']

text = tf.strings.lower(text)

tokenizer(text)

In [ ]:
corpus_dataset = Dataset.from_tensor_slices(corpus).batch(2)


corpus_vector = corpus_dataset.map(lambda batch_text: tokenizer.call(batch_text))


In [ ]:
list(corpus_vector)[0]

In [ ]:

class SpaceTokenizer(tf.keras.Model):
    """
    使用 TextVectorization 构建的空格分词器

    Author: xrh
    Date: 2021-12-15

    ref:
    https://www.tensorflow.org/text/guide/subwords_tokenizer

    """

    def __init__(self, corpus):
        """

        """

        super(SpaceTokenizer, self).__init__(self)
        
        self.tokenizer = TextVectorization(
            standardize=None,
            output_sequence_length=50,
            max_tokens=20000)

        self.tokenizer.adapt(corpus)

        self.vocab_list = self.tokenizer.get_vocabulary()
        

    # @tf.function
    @tf.function(input_signature=[tf.TensorSpec(dtype=tf.string, shape=[None])])
    def tokenize(self, strings):
        """
        返回序列长度(self.max_seq_length)为固定的 tensor

        :param strings:
        :return:
        """
        enc = self.tokenizer.call(strings)
        
        ragged = tf.RaggedTensor.from_tensor(enc, padding=0)

        return ragged

model_tokenizer = SpaceTokenizer(corpus) 

corpus_vector = corpus_dataset.map(lambda batch_text: model_tokenizer.tokenize(batch_text))

list(corpus_vector)[0]

In [ ]:
tokenizer_path = os.path.join('dataset', 'tokenizer_source_model')

tf.saved_model.save(model_tokenizer, tokenizer_path)

model_tokenizer = tf.saved_model.load(tokenizer_path)

corpus_vector = corpus_dataset.map(lambda batch_text: model_tokenizer.tokenize(batch_text))

list(corpus_vector)[0]

#####  StringLookup (字典)

In [ ]:


token_string_from_index = StringLookup(
            vocabulary=vocab_list,
            mask_token='',
            invert=True)

index_seq = [
    [115, 12805,  4027,    30,  6743,    79,    14,  1013],
    [115, 12805,  4027,    30,  6743,    79,    14]
]

result_text_tokens = token_string_from_index(index_seq)



result_text = tf.strings.reduce_join(result_text_tokens, axis=1,
                                    separator=' ')

[text.numpy().decode('utf-8') for text in result_text]


# print(result_text)


In [ ]:
index_from_token_string = StringLookup(
            vocabulary=vocab_list,
            mask_token='',
            )

token_string_seq = ['keine', 'befreiende','novelle', 'für','tymoshenko', 'durch', 'das', 'parlament']

index_from_token_string(token_string_seq)

In [ ]:
start_index = index_from_token_string(tf.constant('[start]'))

start_index

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(corpus).batch(256)


In [ ]:
list(dataset.as_numpy_iterator())[0][0]

In [ ]:

seq = list(dataset.as_numpy_iterator())[0][0]

seq_vector = index_from_token_string(seq.decode('utf-8').split(' '))


seq_vector

# seq_vector_pad = tf.keras.preprocessing.sequence.pad_sequences(seq_vector, padding='post')

# seq_vector_pad

###  Subword tokenizers

#### WordPiece( by TF_text )

In [ ]:
def load_corpus_data( corpus_file_dir):
    """
    读取 图片描述文本, 并将它们和对应的图片进行映射

    :param corpus_file_dir: 语料文本

    :return:

    text_data: 所有句子的列表

    """

    with open(corpus_file_dir, encoding='utf-8') as corpus_file:

        lines = corpus_file.readlines()
        text_data = []

        for line in lines:

            sentence = line.strip()

#             sentence = '[START] ' + sentence + ' [END]'

            text_data.append(sentence)

        return text_data
    
target_corpus = load_corpus_data('dataset/newstest2012.de')

target_dataset = Dataset.from_tensor_slices(target_corpus).batch(16)


In [ ]:
target_corpus[:2]

In [ ]:
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab


bert_tokenizer_params=dict(lower_case=False)
reserved_tokens=["", "[UNK]", "[START]", "[END]", "##AT##-##AT##", "&quot;", "&apos;"]

bert_vocab_args = dict(
    # The target vocabulary size
    vocab_size = 5000,
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens=reserved_tokens,
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)

In [ ]:
%%time
target_vocab = bert_vocab.bert_vocab_from_dataset(
    target_dataset.prefetch(buffer_size=tf.data.AUTOTUNE),
    **bert_vocab_args
)

In [ ]:
print(target_vocab[:10])
print(target_vocab[100:110])
print(target_vocab[1000:1010])
print(target_vocab[-10:])

In [ ]:
def write_vocab_file(filepath, vocab):
  with open(filepath, 'w', encoding='utf-8') as f:
    for token in vocab:
      print(token, file=f)

In [ ]:
write_vocab_file('target_vocab.txt', target_vocab)


In [ ]:
# target_tokenizer = tf_text.BertTokenizer('target_vocab.txt', **bert_tokenizer_params)


In [ ]:
for batch_seq in target_dataset.unbatch().batch(2).take(1):
    
    print(batch_seq.numpy())

In [ ]:
# Tokenize the examples -> (batch, word, word-piece)
token_batch = target_tokenizer.tokenize(batch_seq)

# for ex in token_batch.to_list():
#   print(ex)

# Merge the word and word-piece axes -> (batch, tokens)
token_merge_batch = token_batch.merge_dims(-2,-1)

print('token_merge_batch:')

print(token_merge_batch)

# for ex in token_merge_batch.to_list():
#   print(ex)

In [ ]:
# Lookup each token id in the vocabulary.
txt_tokens = tf.gather(target_vocab, token_merge_batch)
# Join with spaces.
tf.strings.reduce_join(txt_tokens, separator=' ', axis=-1)

In [ ]:
words = target_tokenizer.detokenize(token_merge_batch)
tf.strings.reduce_join(words, separator=' ', axis=-1)

##### Tokenizer 的持久化

In [ ]:
class SubwordTokenizer(tf.keras.Model):
    """
    使用 tensorflow_text.BertTokenizer 构建的基于 subword 的分词器

    Author: xrh
    Date: 2021-12-15

    ref:
    https://www.tensorflow.org/text/guide/subwords_tokenizer
    https://www.tensorflow.org/text/api_docs/python/text/BertTokenizer

    """

    def __init__(self, bert_tokenizer_params, fixed_seq_length, reserved_tokens, vocab_list, _start_str, _end_str):
        """

        :param bert_tokenizer_params: bert_tokenizer 的参数(字典形式)
        :param fixed_seq_length: 指定的序列长度
        :param reserved_tokens: 保留的控制字符,
                ["[NULL]", "[UNK]", "[START]", "[END]"]
        :param vocab_list: 词典
        :param _start_str: 句子的开始
        :param _end_str: 句子的结束
        """

        super(SubwordTokenizer, self).__init__(self)

        self._reserved_tokens = reserved_tokens
        self.fixed_seq_length = fixed_seq_length
        self.start = tf.constant(reserved_tokens.index(_start_str), dtype=tf.int64)
        self.end = tf.constant(reserved_tokens.index(_end_str), dtype=tf.int64)

        lookup = tf.lookup.StaticVocabularyTable(
            num_oov_buckets=1,
            initializer=tf.lookup.KeyValueTensorInitializer(
                keys=vocab_list,
                values=tf.range(len(vocab_list), dtype=tf.int64)))

        # bert_tokenizer_params = dict(lower_case=False)

        self.tokenizer = tf_text.BertTokenizer(lookup, **bert_tokenizer_params)

        self.vocab = tf.Variable(vocab_list)

        # Include a tokenize_fixed signature for a batch of strings.
        self.tokenize_fixed.get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string))

        # Include a tokenize signature for a batch of strings.
        self.tokenize.get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string))

        # Include `detokenize` and `lookup` signatures for:
        #   * `Tensors` with shapes [tokens] and [batch, tokens]
        #   * `RaggedTensors` with shape [batch, tokens]
        self.detokenize.get_concrete_function(
            tf.TensorSpec(shape=[None, None], dtype=tf.int64))

        self.detokenize.get_concrete_function(
            tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

        self.lookup.get_concrete_function(
            tf.TensorSpec(shape=[None, None], dtype=tf.int64))

        self.lookup.get_concrete_function(
            tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    def add_start_end(self, batch_ragged):
        """
        在句子的首尾添加控制字符

        :param batch_ragged:
        :return:
        """
        N_batch = batch_ragged.bounding_shape()[0]
        starts = tf.fill([N_batch, 1], self.start)
        ends = tf.fill([N_batch, 1], self.end)

        return tf.concat([starts, batch_ragged, ends], axis=1)

    # @tf.function(input_signature=[tf.TensorSpec(dtype=tf.string, shape=[None]), tf.TensorSpec(dtype=tf.int64, shape=None)])
    # TODO: 报错 TypeError: Dimension value must be integer or None
    @tf.function
    def tokenize_fixed(self, strings):
        """
        返回固定序列长度(fixed_seq_length)的 tensor

        :param strings:
        :return:
        """

        enc = self.tokenizer.tokenize(strings)

        # Merge the `word` and `word-piece` axes.
        enc = enc.merge_dims(-2, -1)
        enc = self.add_start_end(enc)

        return enc.to_tensor(shape=[None, self.fixed_seq_length])

    @tf.function
    def tokenize(self, strings):
        """
        返回不规则的 tensor(RaggedTensor)

        :param strings:
        :return:
        """

        enc = self.tokenizer.tokenize(strings)

        # Merge the `word` and `word-piece` axes.
        enc = enc.merge_dims(-2, -1)
        enc = self.add_start_end(enc)

        return enc

    @tf.function
    def detokenize(self, tokenized):
        """
        合并所有的 subword 并返回单词列表

        :param tokenized:
        :return:
        """
        words = self.tokenizer.detokenize(tokenized)
        return words

    @tf.function
    def lookup(self, token_ids):
        """
        返回 subword 的 token 列表

        :param token_ids:
        :return:
        """
        return tf.gather(self.vocab, token_ids)


In [ ]:
# reserved_tokens=["", "[UNK]", "[START]", "[END]", "##AT##-##AT##", "&quot;", "&apos;"]

fixed_seq_length = 50

tokenizer_obj = SubwordTokenizer(bert_tokenizer_params, fixed_seq_length, reserved_tokens, target_vocab,
                             _start_str="[START]", _end_str="[END]")

model_name = 'target_tokenizer.bin'

tf.saved_model.save(tokenizer_obj, model_name)

In [ ]:

tf.constant(reserved_tokens.index("[START]"), dtype=tf.int64)

tf.argmax(tf.constant(reserved_tokens) == "[START]")


In [ ]:
model_name = 'target_tokenizer.bin'
tokenizer_obj = tf.saved_model.load(model_name)
tokenizer_obj

In [ ]:
for batch_seq in target_dataset.unbatch().batch(2).take(1):
    
    print(batch_seq.numpy())

In [ ]:
max_seq_length = tf.constant(50)

print(max_seq_length)

token_batch = tokenizer_obj.tokenize_fixed(batch_seq)

token_batch

# token_batch.to_tensor()

In [ ]:
batch_seq=['Die CDU schlägt vor , den EU ##AT##-##AT## Präsidenten aufgrund allgemeiner Stimmabgabe zu wählen',
          '&quot; Ethisches Gebot &quot;'
          ]

token_batch = tokenizer_obj.tokenize(batch_seq)

token_batch

In [ ]:
words = tokenizer_obj.detokenize(token_batch)

# words

texts = tf.strings.reduce_join(words, separator=' ', axis=-1).numpy()

texts

texts_utf8 = [sentence.decode('utf-8') for sentence in texts]

texts_utf8

In [ ]:
words = tokenizer_obj.lookup(token_batch)

words

##### bug: reserved token  不 work

In [ ]:
vocab_list = [
    # reserved tokens
    b"[UNK]", "[MASK]", b"[RANDOM]", b"[CLS]", b"[SEP]",
    # Suffixes
    b"Sponge", b"&quot;",
    b"##ack", b"##ama", b"##ger", b"##gers", b"##onge", b"##pants",  b"##uare",
    b"##vel", b"##ven", b"an", b"A", b"Bar", b"Hates", b"Mar", b"Ob",
    b"Patrick", b"President", b"Sp", b"Sq", b"bob", b"box", b"has", b"highest",
    b"is", b"office", b"the",
]
# TODO: vocab_list 中, 若 token 的前后含有符号则在 tokenize 时无法识别( eg. &quot; , [MASK]), 原因未知 

lookup_table = tf.lookup.StaticVocabularyTable(
    tf.lookup.KeyValueTensorInitializer(
      keys=vocab_list,
      key_dtype=tf.string,
      values=tf.range(
          tf.size(vocab_list, out_type=tf.int64), dtype=tf.int64),
      value_dtype=tf.int64),
      num_oov_buckets=1
)



In [ ]:
sentences = ["[MASK] Sponge &quot; bob Squarepants [CLS] is an Avenger"]

bert_tokenizer_params=dict(lower_case=False)

bert_tokenizer = tf_text.BertTokenizer(vocab_lookup_table=lookup_table, 
                                    token_out_type=tf.int64, **bert_tokenizer_params
                                   )

segments = bert_tokenizer.tokenize(sentences)

# print(segments)

segments = segments.merge_dims(-2, -1)

print(segments)


detoken = bert_tokenizer.detokenize(segments)
sentence = tf.strings.reduce_join(detoken, separator=' ', axis=-1)

print(sentence)

In [ ]:

bert_tokenizer_string = tf_text.BertTokenizer(
                             vocab_lookup_table=lookup_table, 
                             token_out_type=tf.string,
                             preserve_unused_token= True
                            )

segments = bert_tokenizer_string.tokenize(sentences)

print(segments)

segments = segments.merge_dims(-2, -1)

print(segments)



#### BPE ( by Hugging)

 Byte Pair Encoding
 

In [ ]:
from tokenizers import Tokenizer

from tokenizers import ByteLevelBPETokenizer

from tokenizers.models import BPE

from tokenizers.trainers import BpeTrainer

from tokenizers.pre_tokenizers import Whitespace

from tokenizers import decoders



##### BPE

 **ref:**
 
 https://huggingface.co/docs/tokenizers/python/v0.10.0/quicktour.html

In [ ]:

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

reserved_tokens=["", "[UNK]", "[START]", "[END]", "##AT##-##AT##", "&quot;", "&apos;"]

trainer = BpeTrainer(special_tokens=reserved_tokens, vocab_size=5000)

tokenizer.pre_tokenizer = Whitespace()

files = ['dataset/newstest2012.de']

tokenizer.train(files, trainer)

tokenizer.save("dataset/cache data/tokenizer-de.json")

tokenizer = Tokenizer.from_file("dataset/cache data/tokenizer-de.json")


##### ByteLevelBPETokenizer

字节级 BPE 将所有 Unicode 码位转换为多个字节级字符：

1.每个 Unicode 码位分解为字节（ASCII 字符为 1 个字节，UTF-8 字符最多为 4 个字节）

2.每个字节的值都会从 Unicode 表的开头获取 一个"可见"字符。这一点尤其重要，因为有很多控制字符，因此我们不能只使用简单的映射： ASCII 字符<->字节值。
因此，某些字符看起来就不一样了，例如空格 U+0020 变为 Ġ。

目的是，通过这样做，你最终会得到一个包含256个 token 的初始字母表。
然后，这 256 个标记可以合并在一起，以表示词汇表中的任何其他token。这么做使得词汇表较小，并且不需要标记unk的 token。

**ref:**

[1] https://github.com/huggingface/tokenizers/issues/203


In [ ]:
files = ['dataset/newstest2012.de']

reserved_tokens=["", "[UNK]", "[START]", "[END]", "##AT##-##AT##", "&quot;", "&apos;"]

tokenizer = ByteLevelBPETokenizer(lowercase=False)

tokenizer.train(files=files, vocab_size=5000, special_tokens=reserved_tokens)

tokenizer.save("dataset/cache data/tokenizer-de.json")

tokenizer = Tokenizer.from_file("dataset/cache data/tokenizer-de.json")


In [ ]:


batch_seq=['Die CDU schlägt vor , den EU ##AT##-##AT## Präsidenten aufgrund allgemeiner Stimmabgabe zu wählen',
          ' [START] &quot; Ethisches Gebot &quot; [END]'
          ]

for seq in batch_seq:

    output = tokenizer.encode(seq)

    print(output.tokens)
    print(output.ids)



In [ ]:
encode_list = tokenizer.encode_batch(batch_seq)

for ele in encode_list:
    
    print(ele.tokens)
    print(ele.ids)

In [ ]:

decode1=tokenizer.decode(encode_list[0].ids, skip_special_tokens=False)

print(decode1)

decode2=tokenizer.decode(encode_list[1].ids, skip_special_tokens=False)

print(decode2)

In [ ]:
tokenized = [
    [414, 424, 42, 59, 417, 870, 626, 452, 269, 340, 1511, 227, 4, 2351, 2319, 3782, 276, 264, 3429, 443, 1641, 327, 278, 4208],
    [227, 2, 227, 5, 2570, 78, 1855, 1048, 294, 227, 5, 227, 3]
]


tokenizer.decode_batch(tokenized, skip_special_tokens=False)


##### 与 tf.dataset 结合

In [ ]:
target_dataset = target_dataset.unbatch().batch(2)

for i,element in enumerate(target_dataset.take(1)):
    
    print(element.numpy())

    
# target_dataset.map(lambda target: tokenizer.encode_batch(target) )
                                      

In [ ]:
class BPETokenizer(tf.keras.Model):
    """
    使用  ByteLevelBPETokenizer 构建的基于 subword 的分词器

    Author: xrh
    Date: 2021-12-15

    ref:
    https://huggingface.co/docs/tokenizers/python/v0.10.0/pipeline.html

    """

    def __init__(self, corups_file, reserved_tokens, vocab_size, _start_str, _end_str):
        """


        :param reserved_tokens: 保留的控制字符,
                ["[NULL]", "[UNK]", "[START]", "[END]"]
        :param vocab_size: 词典大小
        :param _start_str: 句子的开始
        :param _end_str: 句子的结束
        """

        super(BPETokenizer, self).__init__(self)

        self._reserved_tokens = reserved_tokens
        
        self.start = tf.constant(reserved_tokens.index(_start_str), dtype=tf.int64)
        self.end = tf.constant(reserved_tokens.index(_end_str), dtype=tf.int64)

        self.tokenizer = ByteLevelBPETokenizer(lowercase=False)

        self.tokenizer.train(files=corups_file, vocab_size=vocab_size, special_tokens=reserved_tokens)


#         self.tokenize.get_concrete_function(
#             tf.TensorSpec(shape=[None], dtype=tf.string))


#         self.detokenize.get_concrete_function(
#             tf.TensorSpec(shape=[None, None], dtype=tf.int64))

#         self.detokenize.get_concrete_function(
#             tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))


    def add_start_end(self, batch_ragged):
        """
        在句子的首尾添加控制字符

        :param batch_ragged:
        :return:
        """
        N_batch = batch_ragged.bounding_shape()[0]
        starts = tf.fill([N_batch, 1], self.start)
        ends = tf.fill([N_batch, 1], self.end)

        return tf.concat([starts, batch_ragged, ends], axis=1)


#     @tf.function
    def tokenize(self, strings):
        """
        返回不规则的 tensor(RaggedTensor)

        :param strings:
        :return:
        """
        strings = [ sentence.decode('utf-8') for sentence in strings.numpy() ] 
        
        print(strings)
        
        encode_list = self.tokenizer.encode_batch(strings)
        
        ragged = tf.ragged.constant([ enc.ids for enc in encode_list])
#         enc = self.add_start_end(enc)

        return ragged

#     @tf.function
    def detokenize(self, tokenized):
        """
        合并所有的 subword 并返回单词列表

        :param tokenized:
        :return:
        """
        
        tokenized = tokenized.to_list()
        
        sentence_list = self.tokenizer.decode_batch(tokenized, skip_special_tokens=False)
        
        words = [sentence.split() for sentence in sentence_list]
        
        return words



files = ['dataset/newstest2012.de']

reserved_tokens=["", "[UNK]", "[START]", "[END]", "##AT##-##AT##", "&quot;", "&apos;"]

    
tokenizer_obj = BPETokenizer(corups_file=files, reserved_tokens=reserved_tokens, vocab_size=5000, _start_str="[START]", _end_str="[END]") 


In [ ]:
batch_seq=tf.constant(
         ['Die CDU schlägt vor , den EU ##AT##-##AT## Präsidenten aufgrund allgemeiner Stimmabgabe zu wählen',
          ' [START] &quot; Ethisches Gebot &quot; [END]'
          ])

vectors = tokenizer_obj.tokenize(batch_seq)

vectors

In [ ]:
words = tokenizer_obj.detokenize(vectors)

words

##### 原理

In [ ]:

import re, collections

def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

vocab = {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w e s t </w>': 6, 'w i d e s t </w>': 3}
num_merges = 1000
for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print(best)
    
    print(vocab)


In [ ]:
vocab

#### WordPiece(by Hugging)

**ref:**

[1] https://huggingface.co/docs/tokenizers/python/v0.10.0/pipeline.html

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase, NFD, StripAccents
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import WordPieceTrainer
from tokenizers import decoders

wordpiece_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))

wordpiece_tokenizer.normalizer = normalizers.Sequence([NFD(), Lowercase(), StripAccents()])

wordpiece_tokenizer.pre_tokenizer = Whitespace()

reserved_tokens=["", "[UNK]", "[START]", "[END]", "##AT##-##AT##", "&quot;", "&apos;"]


trainer = WordPieceTrainer(
    vocab_size=5000, special_tokens=reserved_tokens
)

files = ['dataset/newstest2012.de']

wordpiece_tokenizer.train(files, trainer)

wordpiece_tokenizer.save("dataset/cache data/tokenizer-de.json")

wordpiece_tokenizer = Tokenizer.from_file("dataset/cache data/tokenizer-de.json")


In [ ]:
batch_seq=['Die CDU schlägt vor , den EU ##AT##-##AT## Präsidenten aufgrund allgemeiner Stimmabgabe zu wählen',
          ' [START] &quot; Ethisches Gebot &quot; [END]'
          ]

for seq in batch_seq:

    res_encode = wordpiece_tokenizer.encode(seq)

    print(res_encode.tokens)
    print(res_encode.ids)

In [ ]:
encode_list = wordpiece_tokenizer.encode_batch(batch_seq)

for ele in encode_list:
    
    print(ele.tokens)
    print(ele.ids)


In [ ]:

wordpiece_tokenizer.decoder = decoders.WordPiece()

wordpiece_tokenizer.decode(ele.ids, skip_special_tokens=False)




In [ ]:
batch_vec = [
    [114, 36, 562, 1620, 76, 207, 15, 159, 1078, 4, 1992, 1669, 2418, 107, 1956, 233, 1412, 146, 2104],
    [2, 5, 1815, 83, 1541, 567, 128, 5, 3]
]

wordpiece_tokenizer.decode_batch(batch_vec, skip_special_tokens=False)



## 不规则 Tensor(RaggedTensor)

In [ ]:
# b=[[1, 1, 1, 1, 1],
#    [2, 2, 2, 2]
# ]

# b_tensor = tf.constant(b)

# print(b_tensor)

### 和 tensor 的转换

In [ ]:

batch_text = tf.RaggedTensor.from_row_splits(
      values=[3, 1, 4, 1, 5, 9, 2, 6],
      row_splits=[0, 4, 4, 7, 8, 8])

batch_text

batch_text.to_tensor()

# target_out = batch_text.map(lambda text: text[1:])

# target_in = batch_text.map(lambda text: text[ :-1])


In [ ]:
target_out

In [ ]:
digits = tf.ragged.constant([[3, 1, 4, 1], [], [5, 9, 2], [6], []])

tensor = digits.to_tensor(shape=[None, None])

tensor

tensor = digits.to_tensor(shape=[None, 10])

tensor

In [ ]:
tf.RaggedTensor.from_tensor(tensor, padding=0)

### 维度合并 (merge_dims)

In [ ]:
rt = tf.ragged.constant([[[1, 2], [3]], 
                         [[4, 5, 6]]]
                       )
print(rt.merge_dims(0, 1))

print(rt.merge_dims(1, 2))

print(rt.merge_dims(0, 2))


### 操作每一个元素

In [ ]:
rt = tf.ragged.constant([[1, 2, 3], [], [4, 5], [6]])

tf.ragged.map_flat_values(tf.ones_like, rt)

tf.ragged.map_flat_values(tf.multiply, rt, rt)

tf.ragged.map_flat_values(tf.add, rt, 5)


In [ ]:
tf.map_fn(fn=lambda t: tf.range(t, t + 3), elems=tf.constant([3, 5, 2]))



# 建立模型

## 序列（Sequential）建模

In [ ]:
model = Sequential()

model.add( Flatten(input_shape=[28,28]) )
model.add( Dense(100, activation='relu') )
model.add( Dense(10, activation='softmax') )

In [ ]:
model.summary()

In [ ]:
model = Sequential( [Flatten( input_shape=[28, 28] ),
                     Dense( 100, activation='relu' ),
                     Dense( 10, activation='softmax' )
                    ] )

In [ ]:
model.summary()

In [ ]:
model.layers[1].name

In [ ]:
weights, biases = model.layers[1].get_weights()

In [ ]:
weights

In [ ]:
biases

In [ ]:
print( weights.shape )
print( biases.shape )

## 函数（Functional）建模

### 例子1 

In [ ]:
inputs = Input( shape=[28,28] )
x = Flatten()( inputs )
x = Dense(100, activation='relu')(x)
output = Dense(10, activation='softmax')(x)

basic_model = Model( inputs=[inputs], outputs=[output] )

In [ ]:
basic_model.summary()


### 例子2

In [ ]:

def base_model(m):

    # 建立模型第一层的 keras Tensor，只能是Input 或者从 别的地方传来的 tensor from keras.layer
    X = Input(shape=(32,), name='X') # 此时 shape(None,32) ；真正训练时为 shape(m,32)

    init_state = Input(shape=(32,), name='init_state')

    #1.
    # b= K.zeros((100,32))
    # b = Input( name='input_b',tensor=K.zeros((100,32)))  #<tf.Variable 'Variable:0' shape=(100, 32) dtype=float32>
    # 喂入模型的一个 batch的样本的数目 m 此时是未知的，因此为None ，
    # 所以模型的第一层中，所有和 m 相关的tensor 都要使用 Input，相当于tensorflow的一个占位符，其他方法不work:
    #
    # 反例1:
    # b=K.zeros((init_state.shape[0],32)) # init_state.shape[0]=None，K.zeros 的shape参数 中不能出现None
    #
    # 反例2:
    # b= K.zeros((m,32)) # m 由参数传入，m=100
    # b = Input( name='input_b',tensor=K.zeros((m,32))) # <tf.Variable 'Variable:0' shape=(100, 32) dtype=float32>
    # 虽然成功建立的 tensor 但是和 来源于 keras.layer 的a (shape: (None,32)) 无法进行 axis=1的拼接
    # 即报错： ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 32), (100, 32)]

    #2.
    # 连接的所有tensor 必须来源于 keras.layer（血统纯正），否则无法生成计算图
    # 下面两种方法都报 AttributeError: 'NoneType' object has no attribute '_inbound_nodes'
    #M1
    # init_state_2 = K.constant(np.zeros((1,64)), dtype='float32') 
    #M2
    # init_state_2=K.ones((1,64))
    
    #M3 使用 keras.layer.Reshape 转换，但是维度不对,会自动的在全面加上一维
    

    concat = Concatenate(axis=1)([X, init_state])

#     concat2= Concatenate(axis=1)([concat, b])
#     concat2 = Concatenate(axis=0)([concat, init_state_2])

    dense = Dense(1, name='dense_1')(concat)

    print(dense)
    
    model= Model(inputs=[ X , init_state ], outputs=dense)

    return model


M=100
model = base_model(M)



In [ ]:
X=np.random.randint(0,10,size=[M,32])
init_state=np.zeros((M,32))
# b_train=K.zeros((M,32)) #ValueError: If your data is in the form of symbolic tensors, you should specify the `steps_per_epoch` argument (instead of the `batch_size` argument, because symbolic tensors are expected to produce batches of input data).

x_train = [X, init_state]
y_train=np.random.randint(low=0,high=2,size=[M,1])


model.compile(loss='binary_crossentropy', optimizer='sgd')

batch_size=10
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=5
          )

## 子类化（Subclassing）建模

官方教程

https://keras.io/guides/making_new_layers_and_models_via_subclassing/


可训练的权重变量(有状态)必须作为类的变量并且被 add_weight(), 如果嫌麻烦也可以直接在 init 中写 Keras layer

### 例子1

In [ ]:
w = tf.Variable([[1.], [2.]])
x = tf.constant([[3., 4.]])

tf.matmul(w, x)  # 矩阵乘法 相当于 np.dot()

tf.sigmoid(w + x)


In [ ]:
class Linear(Layer):
    
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        ) # 可训练的权重变量(有状态) 作为类的变量并且被 add_weight()
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

N = 2
    
x = tf.ones((N, 2))

linear_layer = Linear(4)

y = linear_layer(x)

print(y)

### 例子2

把 Keras Layer 当做类变量, 与函数建模很类似，好处是： 不用写 输入层(Input) 和 在最后定义 model 指明输入输出层


In [ ]:
class Linear(Layer):  
  
    def __init__( self, units=32, **kwargs ):

        super().__init__(**kwargs)
        self.dense_layer = Dense(units) 

    def call(self, inputs):

        x = Flatten()(inputs)
        x = self.dense_layer(x)
        return x

In [ ]:
model = Linear(4)

model

# model.layers

In [ ]:
x = tf.ones((2, 2))
linear_layer = Linear(4)
y = linear_layer(x)
print(y)

**Layer 和 Model 的区别**


通常，您会使用 Layer 类来定义内部计算块，并使用 Model 类来定义外部模型，即您将训练的对象。

例如，在 ResNet50 模型中，您会有几个子类化 Layer 的 ResNet 块，以及一个包含整个 ResNet50 网络的 Model。

Model 类具有与 Layer 相同的 API，但有如下区别：

* 它会公开内置训练、评估和预测循环（model.fit()、model.evaluate()、model.predict()）。
* 它会通过 model.layers 属性公开其内部层的列表。
* 它会公开保存和序列化 API（save()、save_weights()…）

实际上，Layer 类对应于我们在文献中所称的“层”（如“卷积层”或“循环层”）或“块”（如“ResNet 块”或“Inception 块”）。

同时，Model 类对应于文献中所称的“模型”（如“深度学习模型”）或“网络”（如“深度神经网络”）。

因此，如果您想知道“我应该用 Layer 类还是 Model 类？”，请问自己：我是否需要在它上面调用 fit()？我是否需要在它上面调用 save()？如果是，则使用 Model。如果不是（要么因为您的类只是更大系统中的一个块，要么因为您正在自己编写训练和保存代码），则使用 Layer。

### 例子3


In [ ]:
#   By XRH in 2019.12.28
# 使用 keras.layer 搭建神经网络 不太灵活：
# （1）keras.layer 提供的结构太少
# （2）要使用 后端提供的丰富的函数 还需要使用 Lambda层进行包装，较繁琐
# 因此，我们 使用 后端的函数来 定义属于自己的层


class ConcatDense(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(ConcatDense, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # 为该层创建一个可训练的权重
        self.w = self.add_weight(name='w',
                                      shape=(input_shape[0][1]+input_shape[1][1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        
        self.b = self.add_weight( name='b',
                                    shape=(self.output_dim,),
                                        initializer='zeros',
                                        trainable=False)
        
        self.input_shapes=input_shape
        
        super(ConcatDense, self).build(input_shape)  # 一定要在最后调用它

    def call(self, inputs):
        assert isinstance(inputs, list)
        
        print('input_shapes:', self.input_shapes)
        
        X, init_state = inputs # X shape: (None,32) , init_state shape: (None,32)
        
        print(X)
        print(init_state)
        
        concat=K.concatenate([X,init_state],axis=1) # shape(None, 64)
        print(concat) 
        
        z = K.dot(concat, self.w) + self.b
        # concat shape (None, 64), self.w shape(64, 10) -> shape (None, 10)
        print(z)
        
        a = relu(z) # 激活函数
        print(a)
        
        return a

    def compute_output_shape(self, input_shape):
        assert isinstance(input_shape, list)
        shape_X, shape_init_state = input_shape
        return (shape_X[0], self.output_dim)


    
def test_model():

    # 建立模型第一层的 keras Tensor，只能是Input 或者从 别的地方传来的 tensor from keras.layer
    X = Input(shape=(32,), name='X') # 此时 shape(None,32) ；真正训练时为 shape(m,32)

    init_state = Input(shape=(32,), name='init_state')

    layer1 = ConcatDense(10, name='layer1')([ X , init_state] )
    
    layer2 = Dense(1, name='layer2')(layer1)
    
    model= Model(inputs=[ X , init_state ], outputs=layer2)

    return model


model = test_model()



In [ ]:

N=10 # 10 个样本

X=np.random.randint(0,10,size=[N,32])
init_state=np.zeros((N,32))

x_train = [X, init_state]
y_train=np.random.randint(low=0,high=2,size=[N,1]) # 取值范围 [0, 2)

# y_train

model.compile(loss='binary_crossentropy', optimizer='sgd')

batch_size=10
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=5
          )

# 模型调试

## tensor 的直接计算

### Session

tensoflow 1.x  采用静态图的方法，需要 使用会话来 run 计算图，其才会执行

In [ ]:
import tensorflow.compat.v1 as tf1
import numpy as np 

tf1.disable_v2_behavior()
 
 
a=np.array(
[[1,2,3,4,5],
[1,2,3,4,5],
[1,2,3,4,5]]    
)
# a=np.array([[2.29982214e-10,1.05035841e-03,1.04089566e-04,9.98845458e-01,
#   5.10124494e-08,5.03688757e-10,2.52189380e-10,1.18713073e-09,
#   2.30988277e-08,2.21948682e-08,1.48340121e-07]])



input = tf1.constant(a)
k = 3
output = tf1.nn.top_k(input, k).indices


# one_hot=one_hot_tensor(output,11)
# one_hot=one_hot[0]
# # one_hot[0].shape
# one_hot=K.reshape(one_hot,(1,one_hot.shape[0],one_hot.shape[1]))
# # one_hot.shape
# one_hot_permute=K.permute_dimensions(one_hot,(1,0,2))
# one_hot_permute.shape

with tf1.Session() as sess:
    
    print(sess.run(input))
    print(sess.run(output))
#     print(sess.run(one_hot))
#     print(sess.run(one_hot_permute))


In [ ]:
output

### Eager

TensorFlow 2.X 的Eager模式，也可以看做是动态图模型。该模型下不需要先构造图，然后再使用Session.run()，而是可以得到即时的反馈。这样在研究和开发时会更加符合直觉。



In [ ]:
import tensorflow as tf

print(tf.__version__)

#检查是否开启 eager 模式, 在使用了tensorflow.compat.v1 后就会将 eager 模式退出
tf.executing_eagerly() 

tf.config.experimental_run_functions_eagerly(True) # 没有起到作用



In [ ]:

rank_2_tensor = tf.constant([[1, 2],
                             [3, 4],
                             [5, 6]], dtype=tf.float16)
print(rank_2_tensor)


In [ ]:

a = tf.constant([[10,10],
                 [11.,1.]])

x = tf.constant([[1.,0.],
                 [0.,1.]])

b = tf.Variable(12.)

c = np.ones((2))

y = tf.matmul(a, x) + b 

print(y)

y2 = y+c

print(y2)

In [ ]:
def top_k_tensor(input, k):
    """
    返回张量 input, 第0个维度的 topk 个元素的标号

    :param input:
    :param k:
    :return:
    """

    return tf.nn.top_k(input,k).indices

a = np.array(
    [[1, 2, 3, 4, 5],
     [1, 2, 2, 2, 2],
     [1, 3, 3, 3, 6]]
)
k = 3

# 静态图下要使用 eval 才能触发计算
print('top_k_tensor: \n', K.eval(top_k_tensor(a,k)))

# 动态图下 直接打印即可
print(top_k_tensor(a,k))

In [ ]:
a= np.array([[1,2,3,4]])
layer1 = Dense(1)

out = layer1(a)
out 
print(out) # 直接执行

##  查看所有层(layer)

In [ ]:
model.summary()

In [ ]:
# 查看所有的 layer 

model.layers



In [ ]:
from tensorflow.keras.utils import plot_model 

plot_model(model)

## 输出指定层的参数

In [ ]:
model.layers[2]

model.layers[2].name

model.get_layer('layer1')

W , b = model.layers[2].get_weights()

np.shape(W)
np.shape(b)


In [ ]:
# 输出该层的所有参数
for weight in model.get_layer('layer1').weights:
    print(weight.name, weight.shape)


In [ ]:
# 输出所有层的所有参数

for i in range(len(model.layers)):
    
    for weight in model.layers[i].weights:
        
        print(weight.name, weight.shape)


In [ ]:
# 查看所有层 和 所有层的输出

for index in range(len(model.layers)):
    print(model.get_layer(index=index).name, model.get_layer(index=index).output_shape)


## 模型中间层的计算结果

In [ ]:
model.layers[2].name

model.layers[2].input

model.layers[2].output

### 使用 K.function()

##### 例子1

In [ ]:

mid_func = K.function( inputs=[ model.layers[0].input, model.layers[1].input ] ,outputs=[model.layers[2].output] )

N=5 # 10 个样本

X=np.random.randint(0,10,size=[N,32])
init_state=np.zeros((N,32))

x_train = [X, init_state]

res = mid_func([X, init_state])

np.shape(res)


**使用K.function()可以快速检查层的正确性**

##### 例子2 LSTM

In [ ]:


N = 2
max_length = 4
n_embedding=20
n_vocab=100
n_a = 10


word_embedding_layer = Embedding(n_vocab, n_embedding, input_length=max_length  , name='word_embedding')
lstm_layer = LSTM(units=n_a, return_sequences=True, return_state=True)


word_list = Input(shape=(max_length))  

mask = (word_list != 0) # shape(N,max_length)

word_embedding = word_embedding_layer(inputs=word_list)

out, state_h, state_c = lstm_layer(inputs=word_embedding, mask=mask)
    

f_out = K.function(inputs=[word_list], outputs=out)

f_mask = K.function(inputs=[word_list], outputs=mask)

f_state_h = K.function(inputs=[word_list], outputs=state_h)

f_state_c = K.function(inputs=[word_list], outputs=state_c)


seq_data = np.ones((N, max_length))


# seq_data =np.random.randint(n_vocab,size=(N, max_length))  

res_out = f_out([seq_data])
res_state_h = f_state_h([seq_data])
res_state_c = f_state_c([seq_data])


res_mask = f_mask([seq_data])


In [ ]:
np.shape(res_out)
res_out

In [ ]:
seq_data

np.shape(res_out)
res_mask

In [ ]:
# 把 LSTM 的时间步拆出来, 一步一步执行

# 1.执行第1个时间步

word_list_t = Input(shape=(max_length))  
mask = (word_list_t != 0) 
word_embedding_t = word_embedding_layer(inputs=word_list_t)
out_t, state_h_t, state_c_t = lstm_layer(inputs=word_embedding_t, mask=mask)

f_out_t = K.function(inputs=[word_list_t], outputs=out_t)

# 测试数据
seq_data_0 = np.expand_dims(seq_data[:,0], axis=1)
# np.shape(seq_data_0)

# 推理
res_out_0 = f_out_t([seq_data_0])
res_out_0

res_out[:,0,:]

In [ ]:
# 2.执行第1个时间步，给 LSTM 一个初始隐藏层状态 h0

word_list_t = Input(shape=(max_length))  
h0 = Input(shape=(n_a))
c0 = Input(shape=(n_a))

mask = (word_list_t != 0) 
word_embedding_t = word_embedding_layer(inputs=word_list_t)
h=h0
c=c0
out_t, h_t, c_t = lstm_layer(inputs=word_embedding_t, mask=mask, initial_state=[h, c])

f_out_t = K.function(inputs=[word_list_t, h0, c0], outputs=out_t)
f_h_t = K.function(inputs=[word_list_t, h0, c0], outputs=h_t)
f_c_t = K.function(inputs=[word_list_t, h0, c0], outputs=c_t)

# 测试数据
seq_data_0 = np.expand_dims(seq_data[:,0], axis=1)
# np.shape(seq_data_0)

data_h = np.ones((N, n_a))
# data_h = np.zeros((N, n_a))
data_c = np.zeros((N, n_a))

# 推理
res_out_0 = f_out_t([seq_data_0, data_h, data_c])
res_out_0

# res_out[:,0,:]


res_h_t = f_h_t([seq_data_0, data_h, data_c])
res_h_t


res_c_t = f_c_t([seq_data_0, data_h, data_c])
res_c_t

### 建立新的 model

使用原来的 layer 建立新的计算图

In [ ]:

mid_model = Model( inputs=[ model.layers[0].input, model.layers[1].input ] ,outputs=[model.layers[2].output] )

N=5 # 10 个样本

X=np.random.randint(0,10,size=[N,32])
init_state=np.zeros((N,32))

x_train = [X, init_state]

res = mid_model.predict([X, init_state])

np.shape(res)

# res


##  Graph  execution

### 与 Eager execution 的区别

In [ ]:
@tf.function
def get_MSE(y_true, y_pred):
  sq_diff = tf.pow(y_true - y_pred, 2)
  return tf.reduce_mean(sq_diff)

y_true = tf.random.uniform([5], maxval=10, dtype=tf.int32)
y_pred = tf.random.uniform([5], maxval=10, dtype=tf.int32)

get_MSE(y_true, y_pred)



In [ ]:
# 关闭构建计算图
tf.config.run_functions_eagerly(True)

get_MSE(y_true, y_pred)

# Don't forget to set it back when you are done.
tf.config.run_functions_eagerly(False)

In [ ]:

@tf.function
def get_MSE(y_true, y_pred):
  print("Calculating MSE!")
  sq_diff = tf.pow(y_true - y_pred, 2)
  return tf.reduce_mean(sq_diff)

error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)


In [ ]:
# Now, globally set everything to run eagerly to force eager execution.
tf.config.run_functions_eagerly(True)

error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)

tf.config.run_functions_eagerly(False)

### 非严格执行

计算图中不是所有的操作都被执行

In [ ]:
tf.gather(1, [1])

In [ ]:
def unused_return_eager(x):
  # Get index 1 will fail when `len(x) == 1`
  tf.gather(x, [1]) # unused 
  return x

try:
  print(unused_return_eager(tf.constant([0.0])))
except tf.errors.InvalidArgumentError as e:
  # All operations are run during eager execution so an error is raised.
  print(f'{type(e).__name__}: {e}')



In [ ]:
@tf.function
def unused_return_graph(x):
  tf.gather(x, [1]) # unused
  return x

# Only needed operations are run during graph exection. The error is not raised.
print(unused_return_graph(tf.constant([0.0])))

### 计算图的多态


**1.计算图的构建**

第一阶段为跟踪（tracing），Function 创建一个新的 tf.Graph，在此阶段 Python代码正常运行，但所有TensorFlow操作（比如两个Tensors 相加）都被推迟执行(它们被 tf.Graph捕获 并用来建立计算图，但是不执行计算图)   

在第二阶段，执行在第一步构建好的计算图（tf.Graph）, 此阶段比跟踪阶段快得多

使用 get_concrete_function 函数可以只执行第一阶段

在跟踪阶段会创建一个tf.Graph 并将其包装在新建立的 concrete function中

Function manages 管理所有的 concrete function，并根据输入的 tensor 来选择一个合适的计算图进行计算



**2.input_signature**

在输入 tensor 的 维度 或 数据类型 改变后，框架会通过 Tracing 代码中的所有计算节点来建立新的计算图，我们可以使用 input_signature 来避免重复建立计算图 


In [ ]:
@tf.function()
def g(x):
  print('Tracing with', x)
  return x


print(g(tf.constant([1, 2, 3])))

print(g(tf.constant([1, 2, 3, 4, 5])))



In [ ]:
print(g.pretty_printed_concrete_signatures())

In [ ]:
@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),))
def g(x):
  print('Tracing with', x)
  return x

# No retrace!
print(g(tf.constant([1, 2, 3])))

print(g(tf.constant([1, 2, 3, 4, 5])))

In [ ]:
@tf.function
def double_func(a):
  print("Tracing with", a)
  return a + a

print("Obtaining concrete trace")
double_strings = double_func.get_concrete_function(tf.constant("a"))

print("Executing traced function")
print(double_strings(tf.constant("a")))
print(double_strings(a=tf.constant("b")))


In [ ]:
print(double_strings)

print(double_strings.structured_input_signature)

print(double_strings.structured_outputs)

In [ ]:
@tf.function
def pow(a, b):
    
    print(a) # Caution! Will only happen once when tracing
    tf.print(a)
    
    return a**b

square = pow.get_concrete_function(a=tf.TensorSpec(None, tf.int32), b=tf.TensorSpec(None, tf.int32))
print(square)

In [ ]:
square(tf.constant(10), tf.constant(3))

# square(10, 2)

#### Python 变量作为参数

In [ ]:
def train_one_step():
  pass

@tf.function
def train(num_steps):
  print("Tracing with num_steps = ", num_steps)
  tf.print("Executing with num_steps = ", num_steps)
  for _ in tf.range(num_steps):
    train_one_step()

print("Retracing occurs for different Python arguments.")
train(num_steps=10)
train(num_steps=20)

print()
print("Traces are reused for Tensor arguments.")
train(num_steps=tf.constant(10))
train(num_steps=tf.constant(20))

In [ ]:

@tf.function
def change_to_tensor(ragged, max_seq_length):
    """
    返回序列长度为固定的 tensor

    :param strings:
    :return:
    """
    tf.print(max_seq_length)
    return ragged.to_tensor(shape=[None, max_seq_length])

ragged = tf.ragged.constant([[3, 1, 4, 1], [], [5, 9, 2], [6], []])

# tensor = ragged.to_tensor(shape=[None, 5])

tensor = change_to_tensor(ragged, 4)

tensor

**一个奇怪的bug**

In [ ]:
a = tf.constant(4)
a.numpy()

In [ ]:
@tf.function(input_signature=[tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int32), tf.TensorSpec(dtype=tf.int32, shape=None)])
def change_to_tensor(ragged, max_seq_length):
    """
    返回序列长度为固定的 tensor

    :param strings:
    :return:
    """
    print(ragged)

    print(max_seq_length)

#     tf.print(ragged)
#     tf.print(max_seq_length.numpy())
    tf.print(max_seq_length)
    
    return ragged.to_tensor(shape=[None, max_seq_length])

ragged = tf.ragged.constant([[3, 1, 4, 1], [], [5, 9, 2], [6], []])

# tf.print(ragged)

tensor = change_to_tensor(ragged, tf.constant(4))

tensor

### 流程控制

In [ ]:
# A simple loop

@tf.function
def f(x):
  while tf.reduce_sum(x) > 1:
    tf.print(x)
    x = tf.tanh(x)
  return x

# f(tf.random.uniform([5]))

# print(tf.autograph.to_code(f.python_function))


In [ ]:
@tf.function
def fizzbuzz(n):
  for i in tf.range(1, n + 1):
    print('Tracing for loop')
    if i % 15 == 0:
      print('Tracing fizzbuzz branch')
      tf.print('fizzbuzz')
    elif i % 3 == 0:
      print('Tracing fizz branch')
      tf.print('fizz')
    elif i % 5 == 0:
      print('Tracing buzz branch')
      tf.print('buzz')
    else:
      print('Tracing default branch')
      tf.print(i)

fizzbuzz(tf.constant(5))
# fizzbuzz(tf.constant(20))

fizzbuzz(tf.constant(5))

In [ ]:
class DynamicRNN(tf.keras.Model):

      def __init__(self, rnn_cell):
            
            super(DynamicRNN, self).__init__(self)
            self.cell = rnn_cell

      @tf.function(input_signature=[tf.TensorSpec(dtype=tf.float32, shape=[None, None, 3])])
      def call(self, input_data):
            
            print('Tracing with',input_data)
            
            # [batch, time, features] -> [time, batch, features]
            input_data = tf.transpose(input_data, [1, 0, 2])
            timesteps =  tf.shape(input_data)[0]
            batch_size = tf.shape(input_data)[1]
            outputs = tf.TensorArray(tf.float32, timesteps)
            state = self.cell.get_initial_state(batch_size = batch_size, dtype=tf.float32)

            for i in tf.range(timesteps):
                output, state = self.cell(input_data[i], state)
                outputs = outputs.write(i, output)
            
            tf.print(outputs.read(0))
            
            return tf.transpose(outputs.stack(), [1, 0, 2]), state

lstm_cell = tf.keras.layers.LSTMCell(units = 6)

rnn = DynamicRNN(lstm_cell)

outputs, state = rnn(tf.random.normal(shape=[2,5,3]))


In [ ]:
outputs, state = rnn(tf.random.normal(shape=[10,10,3]))

outputs, state = rnn(tf.random.normal(shape=[20,10,3]))

### python 对象的副作用

In [ ]:
@tf.function
def f(x):
  print("Traced with", x)
  tf.print("Executed with", x)

f(1)
f(1)
f(2)

#### python 变量和 tf.Variable

In [ ]:
class Model(tf.Module):
  def __init__(self):
    self.v = tf.Variable(0)
    self.counter = 0

  @tf.function
  def __call__(self):
    
    print('in Trace step')
    if self.counter == 0:
      
      tf.print('counter:', self.counter)
      # A python side-effect
      self.counter += 1
      self.v.assign_add(1)

    return self.v

m = Model()

print(m().numpy()) # prints 1, 2, 3

print(m().numpy()) 

In [ ]:
class Model(tf.Module):
  def __init__(self):
    self.v = tf.Variable(0)
    self.counter = tf.Variable(0)

  @tf.function
  def __call__(self):
    
    print('in Trace step')
    if self.counter == 0:
      
      tf.print('counter:', self.counter)
      # A python side-effect
      self.counter.assign_add(1)
        
      self.v.assign_add(1)

    return self.v

m = Model()

print(m().numpy()) # prints 1, 2, 3

print(m().numpy()) 

In [ ]:
class Model(tf.Module):
  def __init__(self):
    self.v = tf.Variable(0)
    self.counter = 0

  @tf.function
  def __call__(self):
    if self.counter == 0:
      # Lifts ops out of function-building graphs
      with tf.init_scope():
        self.counter += 1
        self.v.assign_add(1)

    return self.v

m = Model()

print(m().numpy()) 
print(m().numpy()) 

In [ ]:
@tf.function
def buggy_add():
  return 1 + foo

@tf.function
def recommended_add(foo):
  return 1 + foo

foo = 1
print("Buggy:", buggy_add())
print("Correct:", recommended_add(foo))

print("Updating the value of `foo` to 100!")
foo = 100
print("Buggy:", buggy_add())  # Did not change!
print("Correct:", recommended_add(foo))

In [ ]:
@tf.function
def variable_add():
  return 1 + foo

foo = tf.Variable(1)
print("Variable:", variable_add())

print("Updating the value of `foo` to 100!")
foo.assign(100)
print("Variable:", variable_add())

In [ ]:
class BetterModel:

  def __init__(self):
    self.bias = tf.Variable(0.)
    self.weight = tf.Variable(2.)

@tf.function
def evaluate(model, x):
  return model.weight * x + model.bias

better_model = BetterModel()
x = tf.constant(10.)
print(evaluate(better_model, x))

print("Adding bias!")
better_model.bias.assign_add(5.0)  # Note: instead of better_model.bias += 5
print(evaluate(better_model, x))  # This works!

In [ ]:
@tf.function
def f(x):
  v = tf.Variable(1.0)
  return v

f(1.0)

In [ ]:
class Count(tf.Module):
  def __init__(self):
    self.count = None

  @tf.function
  def __call__(self):
    if self.count is None:
      self.count = tf.Variable(0)
    return self.count.assign_add(1)

c = Count()
print(c())
print(c())

#### iterator

In [ ]:
@tf.function
def buggy_consume_next(iterator):
  
  print('in Trace step')
  tf.print("Value:", next(iterator))

iterator = iter([1, 2, 3])
buggy_consume_next(iterator)
# This reuses the first value from the iterator, rather than consuming the next value.
buggy_consume_next(iterator)
buggy_consume_next(iterator)

In [ ]:
@tf.function
def good_consume_next(iterator):
  # This is ok, iterator is a tf.data.Iterator
  tf.print("Value:", next(iterator))

ds = tf.data.Dataset.from_tensor_slices([1, 2, 3])
iterator = iter(ds)
good_consume_next(iterator)
good_consume_next(iterator)
good_consume_next(iterator)

#### list

In [ ]:
external_list = []

@tf.function
def side_effect(x):
  print('in Trace step')
  external_list.append(x)

side_effect(1)
side_effect(1)
side_effect(1)
# The list append only happened once!
print(external_list)
assert len(external_list) == 1

In [ ]:
side_effect(2)
print(external_list)

In [ ]:

@tf.function
def f(x):
    l = [] # python list 对象
    for i in x:
        l.append(i + 1)    # Caution! Will only happen once when tracing
    return l


# f(tf.constant([1, 2, 3]))


In [ ]:
l = []
@tf.function
def f(x):
  for i in x:
    l.append(i + 1)    # Caution! Will only happen once when tracing
f(tf.constant([1, 2, 3]))
# l
l

In [ ]:
@tf.function
def f(x):
  ta = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
  for i in range(len(x)):
    ta = ta.write(i, x[i] + 1)
  return ta.stack()
f(tf.constant([1, 2, 3]))


### tensor 泄露

In [ ]:
x = None

@tf.function
def leaky_function(a):
  global x
  x = a + 1  # Bad - leaks local tensor
  return a + 2

correct_a = leaky_function(tf.constant(1))

print(correct_a.numpy())  # Good - value obtained from function's returns

try:
  x.numpy()  # Bad - tensor leaked from inside the function, cannot be used here
except AttributeError as expected:
  print(expected)

In [ ]:
class MyClass:

  def __init__(self):
    self.field = None

external_list = []
external_object = MyClass()

def leaky_function():
  a = tf.constant(1)
  external_list.append(a)  # Bad - leaks tensor
  external_object.field = a  # Bad - leaks tensor

leaky_function()

print(external_list)

leaky_function()

print(external_list)

# 编译和拟合模型

## 使用内置的方法

In [ ]:
basic_model.compile( loss='sparse_categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'] )

In [ ]:
basic_model.fit( x_train, y_train, epochs=10, validation_split=0.2 )

In [ ]:
class myCallback( tf.keras.callbacks.Callback ):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.9):
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
model = Sequential()

model.add( Flatten(input_shape=[28,28]) )
model.add( Dense(100, activation='relu') )
model.add( Dense(10, activation='softmax') )

model.compile( loss='sparse_categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'] )

model.fit( x_train, y_train, epochs=20, callbacks=[callbacks] )

## 自定义 fit 

In [ ]:
class CustomModel(keras.Model):
    

#     @tf.function(input_signature=[
#         tf.TensorSpec(shape=(None, None), dtype=tf.int64),
#         tf.TensorSpec(shape=(None, None), dtype=tf.int64),
#     ])
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}



In [ ]:
import numpy as np

# Construct and compile an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Just use `fit` as usual
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=3)

In [ ]:
loss_tracker = keras.metrics.Mean(name="loss")
mae_metric = keras.metrics.MeanAbsoluteError(name="mae")


class CustomModel(keras.Model):
    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute our own loss
            loss = keras.losses.mean_squared_error(y, y_pred)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Compute our own metrics
        loss_tracker(loss)
        mae_metric(y, y_pred)
        return {"loss": loss_tracker.result(), "mae": mae_metric.result()}

    @property
    def metrics(self):
        # We list our `Metric` objects here so that `reset_states()` can be
        # called automatically at the start of each epoch
        # or at the start of `evaluate()`.
        # If you don't implement this property, you have to call
        # `reset_states()` yourself at the time of your choosing.
        return [loss_tracker, mae_metric]


# Construct an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)

# We don't passs a loss or metrics here.
model.compile(optimizer="adam")

# Just use `fit` as usual -- you can use callbacks, etc.
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=5)

## 从头编写训练循环

In [ ]:

tf.get_logger().setLevel('ERROR')

# Prepare the training dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

# Reserve 10,000 samples for validation.
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)



In [ ]:
# Get model
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, name="predictions")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
# Instantiate an optimizer to train the model.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

In [ ]:
import time

epochs = 1
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Update training metric.
        train_acc_metric.update_state(y_batch_train, logits)

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * batch_size))

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False)
        # Update val metrics
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))

# TODO: 在 pycharm 中执行不会 打印 <tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>
    

## 混合精度

In [ ]:
import os
from tensorflow.keras.mixed_precision import experimental as mixed_precision


# os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'    #使用GPU0,1

# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'      #使用CPU


tf.config.list_physical_devices('GPU')

# policy = mixed_precision.Policy('mixed_float16')

policy = mixed_precision.Policy('float32')

mixed_precision.set_policy(policy)

inputs = keras.Input(shape=(784,), name='digits')

num_units = 4096

dense1 = layers.Dense(num_units, activation='relu', name='dense_1')
x = dense1(inputs)
dense2 = layers.Dense(num_units, activation='relu', name='dense_2')
x = dense2(x)

print('x.dtype: %s' % x.dtype.name)
# 'kernel' is dense1's variable
print('dense1.kernel.dtype: %s' % dense1.kernel.dtype.name)

# CORRECT: softmax and model output are float32
x = layers.Dense(10, name='dense_logits')(x)
outputs = layers.Activation('softmax', dtype='float32', name='predictions')(x)
print('Outputs dtype: %s' % outputs.dtype.name)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])


In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

In [ ]:
initial_weights = model.get_weights()

# initial_weights

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=8192,
                    epochs=5,
                    validation_split=0.2)
test_scores = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

### 损失放大

float16 也极少出现下溢的情况。此外，在正向传递中出现下溢的情形更是十分罕见。但是，在反向传递中，梯度可能因下溢而变为零。损失放大就是一个防止出现下溢的技巧。

In [ ]:
x = tf.constant(256, dtype='float16')
(x ** 2).numpy()  # Overflow

x = tf.constant(1e-5, dtype='float16')
(x ** 2).numpy()  # Underflow

In [ ]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)


loss_scale = policy.loss_scale
print('Loss scale: %s' % loss_scale)

In [ ]:
new_policy = mixed_precision.Policy('mixed_float16', loss_scale=1024)
print(new_policy.loss_scale)

## 检查点

# 预测模型

In [ ]:
basic_model.evaluate( x_test, y_test )

In [ ]:
prob = model.predict( x_test[0:1] )
prob

In [ ]:
import numpy as np

np.argmax(prob)

In [ ]:
print( y_test[0] )

In [ ]:
plt.imshow(x_test[0])

In [ ]:
# 训练精度 90% 但是测试精度 87.7%，有过拟合的征兆

## 引进验证集

In [ ]:
data = keras.datasets.fashion_mnist

(x_train_full, y_train_full),(x_test, y_test) = data.load_data()

x_valid, x_train = x_train_full[:5000] / 255.0, x_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [ ]:
class myCallback( tf.keras.callbacks.Callback ):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_acc')>0.9):
      print("\nReached 90% validation accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
model = Sequential()

model.add( Flatten(input_shape=[28,28]) )
model.add( Dense(100, activation='relu') )
model.add( Dense(10, activation='softmax') )

model.compile( loss='sparse_categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'] )

history = model.fit( x_train,
                     y_train,
                     epochs=20, 
                     validation_data=(x_valid, y_valid), 
                     callbacks=[callbacks] )

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D

data = keras.datasets.fashion_mnist

(x_train_full, y_train_full),(x_test, y_test) = data.load_data()

x_train_full = x_train_full.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

x_valid, x_train = x_train_full[:5000] / 255.0, x_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]


model = Sequential([Conv2D(64, (3,3), activation='relu', input_shape=(28,28,1)),
                    MaxPooling2D(2,2),
                    Conv2D(64, (3,3), activation='relu'),
                    MaxPooling2D(2,2),
                    Flatten(),
                    Dense(128, activation='relu'),
                    Dense(10, activation='softmax') ])

model.compile( loss='sparse_categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'] )

history = model.fit( x_train,
                     y_train,
                     epochs=20, 
                     validation_data=(x_valid, y_valid) )

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.figure( figsize=(8,4), dpi=100 )

plt.subplot(1,2,1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
from tensorflow.keras.layers import Dropout

model = Sequential([Conv2D(64, (3,3), activation='relu', input_shape=(28,28,1)),
                    MaxPooling2D(2,2),
                    Conv2D(64, (3,3), activation='relu'),
                    MaxPooling2D(2,2),
                    Flatten(),
                    Dropout(0.5),
                    Dense(128, activation='relu'),
                    Dense(10, activation='softmax') ])

model.compile( loss='sparse_categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'] )

history = model.fit( x_train,
                     y_train,
                     epochs=20, 
                     validation_data=(x_valid, y_valid) )

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.figure( figsize=(8,4), dpi=100 )

plt.subplot(1,2,1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## 自定义 评估函数

In [ ]:
class CustomModel(keras.Model):
    def test_step(self, data):
        # Unpack the data
        x, y = data
        # Compute predictions
        y_pred = self(x, training=False)
        # Updates the metrics tracking the loss
        self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        # Update the metrics.
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}


# Construct an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(loss="mse", metrics=["mae"])

# Evaluate with our custom test_step
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.evaluate(x, y)

# 超参数调优 Keras Tuner

**Ref**

[1] https://zhuanlan.zhihu.com/p/156139224

[2] https://www.tensorflow.org/tutorials/keras/keras_tuner

## 例子1 使用序列建模

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
# from kerastuner.tuners import RandomSearch

import keras_tuner as kt

In [ ]:
print( x_train.shape )
print( x_test.shape )
print( y_train.shape )
print( y_test.shape )

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    model.add( Flatten(input_shape=[28,28]) )
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='relu')) 
    model.add(layers.Dense(10, activation='softmax')) 
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [ ]:
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy', #优化目标
    max_epochs=20, 
    directory='E:\python package\python-project\DeepLearningApp\LearnKeras\logs',
    project_name='image_classify')

# directory 要使用绝对路径, 否则报 UnicodeDecodeError 真TM坑

In [ ]:
tuner.results_summary()

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(x_train,
             y_train, 
             epochs=5, 
             validation_split=0.2, callbacks=[stop_early]) 

## 例子2 使用函数建模

In [ ]:

def build_model2(hp):

    inputs = Input(shape=[28,28])
    x = Flatten(input_shape=[28,28])(inputs)
    x = Dense(units=hp.Int('units',
                            min_value=50,
                            max_value=512,
                            step=50), activation='relu')(x)
    output = Dense(10, activation='softmax')(x)

    model = Model( inputs=[inputs], outputs=[output] )
    
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    
    return model


tuner = kt.Hyperband(
    build_model2,
    objective='val_accuracy', #优化目标
    max_epochs=10, 
    directory='E:\python package\python-project\DeepLearningApp\LearnKeras\logs', # directory 要使用绝对路径, 否则报 UnicodeDecodeError 
    project_name='imageclassify') # 每次运行之前, 将 imageclassify 目录删除

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(x_train,
             y_train, 
             epochs=10, 
             validation_split=0.2, callbacks=[stop_early]) 

In [ ]:
tuner.results_summary(3)

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
tuner.get_best_hyperparameters()[0]

In [ ]:
best_model.evaluate( x_test, y_test )

In [ ]:
best_model.summary()

In [ ]:
basic_model.evaluate( x_test, y_test )

In [ ]:
basic_model.summary()